### 기본 아이디어 : Q러닝을 목표 18개로 나누어서 실행

- Q러닝은 off-policy이므로 exploration / exploitation을 나눠 실행할 수 있다.

- 각각의 목표에 따른 Q-Tables를 만들어서 학습시키고

 Q-Table에 따른 ε-Greedy 정책을 통해서 순차적 task를 수행

- 빠른 exploration을 위해서 시작위치를 랜덤하게 정해줌  
(단순히 랜덤이동이면 시작 위치에서 멀어질수록 탐색확률이 지수적으로 감소하기 때문)
\+ 시작점으로 수렴하는 Q-Table을 만들기 어려우니까

In [6]:
import random
from string import ascii_uppercase
# from draw_utils import * 
# from pyglet.gl import *
import numpy as np
import pandas as pd
import os

In [7]:
# reward
move_reward = -1 
obs_reward = -10     # 한 번 장애물에 부딪치면, 10번에 1번정도만 장애물쪽에 이동하도록 함
goal_reward = 1000   # Q-value의 빠른 수렴을 위해 넉넉하게 리워드를 부여
# print('reward:' , move_reward, obs_reward, goal_reward) 
inds = list(ascii_uppercase)[:17]  # 클래스 활용이 미숙해 전역변수로 변환
inds.append("T")     # T-state (최종 목적지=시작 목적지)를 추가해 줌 (box.csv에도 추가함)
Q_table_name_lst = inds
print(inds)
Q_tables = [] #Q_table_A : np.zero((10, 9, 4)) 형태의 딕셔너리 혹은 Structure Array 형태로 만들어보고 싶었으나 실패
for Q_table_name in Q_table_name_lst:
    globals()["Q_table_{}".format(Q_table_name)] = np.zeros((10, 9, 4)) 
    Q_tables.append(globals()["Q_table_{}".format(Q_table_name)]) 
#일단 완전히 초기화된 Q-Table들의 리스트인 Q-Tables를 만듬, 그냥 np.zeros((18, 10, 9, 4)) 해도 됐을듯 ㅠㅠ
print(len(Q_tables))

local_path = "/content/drive/MyDrive/git_path_finding/path-finding-rl"

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'T']
18


In [40]:
box_data = pd.read_csv(os.path.join(local_path, "./data/box.csv"))

In [47]:
box_coor_index =[]
for i in range(len(box_data)):
    box_coor_index.append([box_data['row'][i], box_data['col'][i]])
print(box_coor_index)

[[5, 0], [4, 0], [3, 0], [2, 0], [0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [2, 8], [3, 8], [4, 8], [5, 8], [9, 4]]


### Q-Tables의 인덱스 = box_coor_index의 인덱스

In [73]:
class Simulator_exploration:  #단순 탐색을 위한 환경
    def __init__(self):
        '''
        height : 그리드 높이
        width : 그리드 너비 
        inds : A ~ Q alphabet list
        '''
        # Load train data
        self.files = pd.read_csv(os.path.join(local_path, "./data/Q_Finder.csv")) #각각 A부터 T까지 하나를 목표로하는 csv파일을 만듬
        self.height = 10 #세로 10
        self.width = 9 # 가로 9  #A~Q까지 알파벳(target) 선언
        self.failcount = 0
        self.successcount = 0
    def set_box(self):
        '''
        아이템들이 있을 위치를 미리 정해놓고 그 위치 좌표들에 아이템이 들어올 수 있으므로 그리드에 100으로 표시한다.
        데이터 파일에서 이번 에피소드 아이템 정보를 받아 가져와야 할 아이템이 있는 좌표만 -100으로 표시한다.
        self.local_target에 에이전트가 이번에 방문해야할 좌표들을 저장한다.
        따라서 가져와야하는 아이템 좌표와 end point 좌표(처음 시작했던 좌표로 돌아와야하므로)가 들어가게 된다.
        '''
        box_data = pd.read_csv(os.path.join(local_path, "./data/box.csv"))

        # 물건이 들어있을 수 있는 경우
        for box in box_data.itertuples(index = True, name ='Pandas'):  #판다스 데이터를 튜플로 iter해준 것 같다. 행/열 위치정보로 각 알파벳이 표시됨
            self.grid[getattr(box, "row")][getattr(box, "col")] = 100   #아이템이 없는 경우 100

        # 물건이 실제 들어있는 경우
        order_item = list(set(inds) & set(self.items))
        order_csv = box_data[box_data['item'].isin(order_item)]
        for order_box in order_csv.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = -100 # 아이템이 있는 경우 -100 <- 값이 지정되면 리턴으로 사용가능?
            # local target에 가야 할 위치 좌표 넣기
            self.local_target.append(
                [getattr(order_box, "row"),
                 getattr(order_box, "col")]
                )# 타겟 위치 지정 완료
            # print(order_box)
            
        # print(self.local_target)
        # self.local_target.append([9,4])  # 마지막 위치를 추가해주지 않음

        # 알파벳을 Grid에 넣어서 -> grid에 2Dconv 적용 가능

    def set_obstacle(self):
        '''
        장애물이 있어야하는 위치는 미리 obstacles.csv에 정의되어 있다. 이 좌표들을 0으로 표시한다.
        '''
        obstacles_data = pd.read_csv(os.path.join(local_path, "./data/obstacles.csv"))
        for obstacle in obstacles_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(obstacle, "row")][getattr(obstacle, "col")] = 0 #장애물 행,열 지정

    def reset(self, epi):
        '''
        reset()은 첫 스텝에서 사용되며 그리드에서 에이전트 위치가 start point에 있게 한다.

        :param epi: episode, 에피소드 마다 가져와야 할 아이템 리스트를 불러올 때 사용
        :return: 초기셋팅 된 그리드
        :rtype: numpy.ndarray
        _____________________________________________________________________________________
        items : 이번 에피소드에서 가져와야하는 아이템들
        terminal_location : 현재 에이전트가 찾아가야하는 목적지
        local_target : 한 에피소드에서 찾아가야하는 아이템 좌표, 마지막 엔드 포인트 등의 위치좌표들
        actions: visualization을 위해 에이전트 action을 저장하는 리스트
        curloc : 현재 위치
        '''

        # initial episode parameter setting
        self.epi = epi
        self.items = list(self.files.iloc[self.epi])[0]
        self.cumulative_reward = 0
        self.terminal_location = None
        self.local_target = []
        self.actions = [] # 정책에 따라 시간 순서대로 action 리스트에 append해주면 될듯

        # initial grid setting
        self.grid = np.ones((self.height, self.width), dtype="float16") #초기는 전부0

        # set information about the gridworld
        self.set_box() #빈박스 + 타겟 박스
        self.set_obstacle() #장애물

        # start point를 grid에 표시
        self.curloc = [np.random.randint(10), np.random.randint(9)] #시작 위치 초기화 (랜덤 위치)
        # obstacle, target 등의 위치를 빼고 시작한다면 더 빠른 수렴이 가능하겠지만, 어차피 그 위치의 q-value는 사용될리 없으므로 완전 랜덤으로 해줌
        self.grid[int(self.curloc[0])][int(self.curloc[1])] = -5       
        
        self.done = False
        
        return self.grid

    def apply_action(self, action, cur_x, cur_y):
        '''
        에이전트가 행한 action대로 현 에이전트의 위치좌표를 바꾼다.
        action은 discrete하며 4가지 up,down,left,right으로 정의된다.
        
        :param x: 에이전트의 현재 x 좌표
        :param y: 에이전트의 현재 y 좌표
        :return: action에 따라 변한 에이전트의 x 좌표, y 좌표
        :rtype: int, int
        '''
        new_x = cur_x
        new_y = cur_y
        # up
        if action == 0:
            new_x = cur_x - 1
        # down
        elif action == 1:
            new_x = cur_x + 1
        # left
        elif action == 2:
            new_y = cur_y - 1
        # right
        else:
            new_y = cur_y + 1

        return int(new_x), int(new_y)


    def get_reward(self, new_x, new_y, out_of_boundary):
        '''
        get_reward함수는 리워드를 계산하는 함수이며, 상황에 따라 에이전트가 action을 옳게 했는지 판단하는 지표가 된다.

        :param new_x: action에 따른 에이전트 새로운 위치좌표 x
        :param new_y: action에 따른 에이전트 새로운 위치좌표 y
        :param out_of_boundary: 에이전트 위치가 그리드 밖이 되지 않도록 제한
        :return: action에 따른 리워드
        :rtype: float
        '''

        # 바깥으로 나가는 경우
        if any(out_of_boundary):
            reward = obs_reward
                       
        else:
            # 장애물에 부딪히는 경우 
            if self.grid[new_x][new_y] == 0:
                reward = obs_reward  

            # 현재 목표에 도달한 경우
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:
                reward = goal_reward

            # 그냥 움직이는 경우 
            else:
                reward = move_reward

        return reward

    def step(self, action):
        ''' 
        에이전트의 action에 따라 step을 진행한다.
        action에 따라 에이전트 위치를 변환하고, action에 대해 리워드를 받고, 어느 상황에 에피소드가 종료되어야 하는지 등을 판단한다.
        에이전트가 endpoint에 도착하면 gif로 에피소드에서 에이전트의 행동이 저장된다.

        :param action: 에이전트 행동
        :return:
            grid, 그리드
            reward, 리워드
            cumulative_reward, 누적 리워드
            done, 종료 여부
            goal_ob_reward, goal까지 아이템을 모두 가지고 돌아오는 finish율 계산을 위한 파라미터

        :rtype: numpy.ndarray, float, float, bool, bool/str

        (Hint : 시작 위치 (9,4)에서 up말고 다른 action은 전부 장애물이므로 action을 고정하는 것이 좋음)
        '''
        # print(self.local_target)
        self.terminal_location = self.local_target[0]
        cur_x,cur_y = self.curloc
        self.actions.append((cur_x, cur_y))

        goal_ob_reward = False
        
        new_x, new_y = self.apply_action(action, cur_x, cur_y)

        out_of_boundary = [new_x < 0, new_x >= self.height, new_y < 0, new_y >= self.width]

        # 바깥으로 나가는 경우 종료
        if any(out_of_boundary):
            self.done = True
            goal_ob_reward = True
            self.failcount += 1 # 실패 카운터
        else:
            # 장애물에 부딪히는 경우 종료
            if self.grid[new_x][new_y] == 0:
                self.done = True
                goal_ob_reward = True
                self.failcount += 1 # 실패 카운터

            # 현재 목표에 도달한 경우, 다음 목표설정
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:
                self.successcount +=1 # 성공 카운터

                # end point 일 때
                # if [new_x, new_y] == [9,4]:
                self.done = True # 목표에 도착하면 바로 done 수행

                # self.local_target.remove(self.local_target[0]) 
                # self.grid[cur_x][cur_y] = 1
                # self.grid[new_x][new_y] = -5
                goal_ob_reward = True
                self.curloc = [new_x, new_y]
            else:
                # 그냥 움직이는 경우 
                self.grid[cur_x][cur_y] = 1
                self.grid[new_x][new_y] = -5
                self.curloc = [new_x,new_y]
                
        reward = self.get_reward(new_x, new_y, out_of_boundary)
        self.cumulative_reward += reward

        # if self.done == True:
        #     if [new_x, new_y] == [9, 4]: #초기위치로 도달했을 때,
        #         if self.terminal_location == [9, 4]: # 목표와 같을때만 gif로 만들어줌
        #             # 완료되면 GIFS 저장
        #             goal_ob_reward = 'finish'
        #             height = 10
        #             width = 9
        #             display = Display(visible=False, size=(width, height))
        #             display.start()
        #
        #             start_point = (9, 4)
        #             unit = 50
        #             screen_height = height * unit
        #             screen_width = width * unit
        #             log_path = "./logs"
        #             data_path = "./data"
        #             render_cls = Render(screen_width, screen_height, unit, start_point, data_path, log_path)
        #             for idx, new_pos in enumerate(self.actions):
        #                 render_cls.update_movement(new_pos, idx+1)
        #
        #             render_cls.save_gif(self.epi)
        #             render_cls.viewer.close()
        #             display.stop()
        # 코랩환경이라 그림그리기는 주석처리함
        return self.grid, reward, self.cumulative_reward, self.done, goal_ob_reward



            


In [17]:
class QAgent_exploration():
    def __init__(self):
        self.eps = 0.0
        self.eps_decay = 0.00
    # 랜덤한 위치 -> 굳이 입실론을 통해 탐색하지 않아도 모든 지점에서 시작할 확률이 같으므로 입실론은 무의미
    
    def select_action(self, curloc, index):
        # Q-table은 단순 그리디로 이동, 목표에 따라 Q-Tables에서 인덱스를 알아야하므로 인덱스를 인수로 추가해줌
        x, y = curloc
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = Q_tables[index][x,y,:] # 목표의 인덱스 = 목표의 Q-Table
            action = np.argmax(action_val)
        return action

    def update_table(self, transition, index):
        s, a, r , s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime, index) 
        Q_tables[index][x,y,a] = Q_tables[index][x,y,a] + 0.1*(r + np.max(Q_tables[index][next_x, next_y, :]) - Q_tables[index][x,y,a])
        #Q러닝 업데이트 식을 이용, 마찬가지로 index를 인수로 추가해 table에 접근함

    def anneal_eps(self):
        self.eps -= self.eps_decay
        self.eps = max(self.eps, 0.1) # 만에하나 한번도 못가본 장소가 있을까봐 0.1만 뒀음 (0이라도 상관없을 듯)
 

In [19]:
def main():
    env = Simulator_exploration()
    agent = QAgent_exploration()
    episode = 10000
    files = pd.read_csv(os.path.join(local_path, "./data/Q_Finder.csv"))

    for n_epi in range(episode): #에피소드 변수만큼 실행

        for targetidx in range(len(files)): # 목표를 A~T까지 돌아가면서 수행 + 업데이트
            items = list(files.iloc[targetidx])[0]
            done = False
            if n_epi % 20 == 0:
                print(f'에피소드 :{n_epi} ')
            env.reset(targetidx)
            while not done: # 한 에피소드가 끝날때 까지
                s = env.curloc
                a = agent.select_action(s, targetidx)
                _, r, cum_reward, done, goal_ob_reward = env.step(a)
                s_prime = env.curloc
                agent.update_table((s, a, r, s_prime), targetidx)
                s = s_prime
            agent.anneal_eps()


In [11]:
main()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
에피소드 :4440, 인덱스 4
에피소드 :4440, 인덱스 5
에피소드 :4440, 인덱스 6
에피소드 :4440, 인덱스 7
에피소드 :4440, 인덱스 8
에피소드 :4440, 인덱스 9
에피소드 :4440, 인덱스 10
에피소드 :4440, 인덱스 11
에피소드 :4440, 인덱스 12
에피소드 :4440, 인덱스 13
에피소드 :4440, 인덱스 14
에피소드 :4440, 인덱스 15
에피소드 :4440, 인덱스 16
에피소드 :4440, 인덱스 17
에피소드 :4460, 인덱스 0
에피소드 :4460, 인덱스 1
에피소드 :4460, 인덱스 2
에피소드 :4460, 인덱스 3
에피소드 :4460, 인덱스 4
에피소드 :4460, 인덱스 5
에피소드 :4460, 인덱스 6
에피소드 :4460, 인덱스 7
에피소드 :4460, 인덱스 8
에피소드 :4460, 인덱스 9
에피소드 :4460, 인덱스 10
에피소드 :4460, 인덱스 11
에피소드 :4460, 인덱스 12
에피소드 :4460, 인덱스 13
에피소드 :4460, 인덱스 14
에피소드 :4460, 인덱스 15
에피소드 :4460, 인덱스 16
에피소드 :4460, 인덱스 17
에피소드 :4480, 인덱스 0
에피소드 :4480, 인덱스 1
에피소드 :4480, 인덱스 2
에피소드 :4480, 인덱스 3
에피소드 :4480, 인덱스 4
에피소드 :4480, 인덱스 5
에피소드 :4480, 인덱스 6
에피소드 :4480, 인덱스 7
에피소드 :4480, 인덱스 8
에피소드 :4480, 인덱스 9
에피소드 :4480, 인덱스 10
에피소드 :4480, 인덱스 11
에피소드 :4480, 인덱스 12
에피소드 :4480, 인덱스 13
에피소드 :4480, 인덱스 14
에피소드 :4480, 인덱스 15
에피소드 :4480, 인덱스 16
에피소드 :4480, 인덱스 17
에피소드 :4500, 인덱스 0
에피소드 :4500, 인덱스 1
에피소드

In [20]:
main()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
에피소드 :9700
성공
에피소드 :9700
성공
에피소드 :9700
성공
에피소드 :9700
성공
에피소드 :9700
에피소드 :9700
에피소드 :9700
에피소드 :9700
성공
에피소드 :9700
성공
에피소드 :9700
에피소드 :9700
성공
에피소드 :9700
성공
에피소드 :9700
성공
에피소드 :9700
성공
에피소드 :9700
성공
에피소드 :9700
성공
에피소드 :9700
에피소드 :9700
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성공
성

In [15]:
import pickle
file = open("/content/drive/MyDrive/git_path_finding/path-finding-rl/data/save.txt", "wb") 
pickle.dump(Q_tables, file)
file.close

<function BufferedWriter.close>

### 텍스트로 Q테이블 저장해보려고 했는데 안됨... 알려주세요

In [24]:
with open('Q_tables(20000).pickle', 'wb') as f:
    pickle.dump(Q_tables, f)

### 피클로 저장할 수 있다고 해서 저장 -> data 폴더에 넣어둠
경로 지정은 어떻게 할지 모르겠습니다.

cd 해서 폴더 위치로 오면 될지도?

In [ ]:
with open('Q_tables(20000).pickle','rb') as f:    #'rb'라는 점을 주의하라
    Q_tables = pickle.load(f)

위치 찾아서 위 테이블을 로드하면 어디서나 사용가능

In [21]:
print(Q_tables)

[array([[[ 7.41193535e+03,  2.23248993e+04,  2.56440010e+03,
          5.67661835e+03],
        [ 5.02245803e+03,  2.24661856e+04,  6.85086923e+03,
          4.49899703e+03],
        [-1.00000000e+00,  2.27796605e+04,  5.69603504e+03,
          1.59629201e+03],
        [ 4.09297375e+03,  2.28185749e+04,  1.80129836e+03,
          6.02408940e+03],
        [ 6.29410213e+03,  2.23749013e+04,  6.65079752e+03,
          7.62515077e+03],
        [ 1.53350589e+04,  2.31885893e+04,  1.46237565e+04,
          8.15016095e+03],
        [ 1.09866767e+04,  1.26812280e+04,  2.28870162e+04,
          8.76490849e+03],
        [ 9.33555855e+03,  5.71151399e+03,  2.25847028e+04,
          8.14759592e+03],
        [ 4.59725491e+03,  3.49239639e+03,  2.14374128e+04,
          5.25510722e+03]],

       [[ 8.76926577e+03,  2.32181820e+04,  4.74463584e+03,
          1.33723288e+04],
        [ 1.04217255e+04,  2.36391839e+04,  7.91763130e+03,
          1.17200374e+04],
        [ 7.13276280e+03,  2.36506160e+0

In [16]:
Q_tables_save_10000 = [array([[[-1.00000000e+00,  1.01630971e+04, -1.00000000e+00,
         -8.04968387e-01],
        [-1.00000000e+00,  1.09551468e+04, -9.05254069e-01,
         -9.54512902e-01],
        [-1.00000000e+00,  1.01987442e+04, -9.99734665e-01,
         -1.05291492e+00],
        [-2.00000000e+00,  1.13754634e+04, -1.22668655e+00,
         -1.21376775e+00],
        [-2.00000000e+00,  1.04215801e+04, -1.45867166e+00,
         -1.39015985e+00],
        [-2.00000000e+00,  1.20982607e+04, -1.65257738e+00,
         -1.74437707e+00],
        [-2.00000000e+00, -1.85965193e+00,  1.17834472e+04,
         -1.95846193e+00],
        [-2.00000000e+00, -2.03660601e+00,  1.12896468e+04,
         -2.00872161e+00],
        [-3.00000000e+00, -2.16149600e+00,  1.01656930e+04,
         -3.00000000e+00]],

       [[-6.26521380e-01,  1.15754890e+04, -1.00000000e+00,
         -6.42211640e-01],
        [-7.95781471e-01,  1.21221055e+04, -7.59653024e-01,
         -7.29086146e-01],
        [-8.89500412e-01,  1.19903733e+04, -8.82623762e-01,
         -9.09910701e-01],
        [-1.11151678e+00,  1.22978702e+04, -1.07690393e+00,
         -1.13569635e+00],
        [-1.35905566e+00, -1.26483663e+00,  1.16797176e+04,
         -1.33842242e+00],
        [-1.64257890e+00,  1.22868061e+04, -1.55957659e+00,
         -1.62985299e+00],
        [-1.81986130e+00,  1.15371228e+04, -1.82242716e+00,
         -1.81998070e+00],
        [-1.97177697e+00, -1.99039737e+00,  1.02109433e+04,
         -2.02514096e+00],
        [-2.13291727e+00,  1.05331805e+04, -2.07900425e+00,
         -3.00000000e+00]],

       [[-4.12610000e-01,  1.22993208e+04, -1.00000000e+00,
         -4.25110000e-01],
        [-5.25900300e-01,  1.27980230e+04, -4.86480000e-01,
         -5.38175490e-01],
        [-7.42455709e-01, -1.00000000e+00,  1.27309395e+04,
         -7.05205970e-01],
        [-9.45188064e-01, -8.94890567e-01,  1.26420453e+04,
         -9.40985237e-01],
        [-1.14347927e+00, -2.00000000e+00,  1.25427172e+04,
         -1.18620157e+00],
        [-1.40684225e+00, -1.32142317e+00,  1.24428991e+04,
         -1.34600994e+00],
        [-1.72458856e+00, -2.00000000e+00,  1.22792538e+04,
         -1.67503759e+00],
        [-1.97773022e+00, -1.93865408e+00,  1.21192346e+04,
         -1.96184745e+00],
        [-2.03403814e+00, -2.09205250e+00,  1.16870201e+04,
         -3.00000000e+00]],

       [[-2.10000000e-01,  1.28501057e+04, -1.00000000e+00,
         -1.00000000e-01],
        [-2.10000000e-01, -1.90000000e-01,  1.28329621e+04,
         -1.00000000e+00],
        [ 1.13006005e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.16880750e+04, -1.01779614e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.16764544e+04, -1.00000000e+00, -1.96915879e-01,
         -1.62812175e-01],
        [ 1.21408330e+04, -1.18180056e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 1.11217495e+04, -1.00000000e+00, -2.00000000e-01,
         -2.52585879e-01],
        [ 1.17673847e+04, -2.03544352e+00, -3.00000000e+00,
         -2.04248952e+00],
        [-2.12252862e+00, -2.12397107e+00,  1.11776698e+04,
         -3.00000000e+00]],

       [[-1.00000000e-01,  1.28555129e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.22672427e+04,
          0.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.11153780e+04,
          0.00000000e+00],
        [-9.84116346e-01,  1.15377800e+04, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.01696906e+04,
          0.00000000e+00],
        [ 1.16141739e+04, -1.33530025e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.06224602e+04,
         -5.52994306e-01],
        [-2.13709987e+00,  1.05965115e+04, -3.00000000e+00,
         -2.09300426e+00],
        [ 1.01043915e+04, -2.20776926e+00, -2.18066462e+00,
         -3.00000000e+00]],

       [[ 1.18561777e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.16766234e+04,
          0.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.00000000e-01,
          1.11105617e+04],
        [-7.91151987e-01,  1.22219071e+04, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.12299452e+04,
          0.00000000e+00],
        [-1.23314478e+00,  1.10036004e+04, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -3.12722702e-01,
          1.04213329e+04],
        [-2.10057887e+00,  1.19578648e+04, -3.00000000e+00,
         -2.10587493e+00],
        [-2.19230510e+00, -2.16002948e+00,  1.09707126e+04,
         -3.00000000e+00]],

       [[ 1.28557166e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.28504213e+04,
          0.00000000e+00],
        [-1.00000000e+00, -1.10000000e-01,  1.16753901e+04,
          0.00000000e+00],
        [-6.02773957e-01,  1.24464964e+04, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.71692350e-01,  1.09780263e+04,
          0.00000000e+00],
        [-1.17184234e+00,  1.22308952e+04, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -2.00188054e-01,  1.11898641e+04,
          0.00000000e+00],
        [-2.01509086e+00,  1.21795477e+04, -2.00000000e+00,
         -1.93119464e+00],
        [-2.00616565e+00,  1.02202116e+04, -1.98898640e+00,
         -2.00000000e+00]],

       [[ 1.24838885e+04, -1.00000000e-01, -1.00000000e+00,
         -1.10000000e-01],
        [ 1.28321226e+04, -2.00000000e-01, -2.00000000e-01,
         -2.10000000e-01],
        [-1.00000000e+00, -4.16470000e-01,  1.27908444e+04,
         -3.97200000e-01],
        [-5.61759000e-01, -5.01368000e-01,  1.27299880e+04,
         -5.10404100e-01],
        [-1.00000000e+00, -8.26942596e-01,  1.26371705e+04,
         -8.34510214e-01],
        [-1.08487327e+00, -1.08924333e+00,  1.25406563e+04,
         -1.11739935e+00],
        [-2.00000000e+00, -1.39159452e+00,  1.24417742e+04,
         -1.33685252e+00],
        [-1.62898630e+00, -1.63336453e+00,  1.23577732e+04,
         -1.63983396e+00],
        [-1.81848508e+00, -1.81097155e+00,  1.17051364e+04,
         -2.00000000e+00]],

       [[ 1.18257901e+04, -1.00000000e+00, -1.00000000e+00,
         -1.10000000e-01],
        [ 1.19680408e+04, -1.00000000e+00, -2.81000000e-01,
         -2.09000000e-01],
        [ 1.22448049e+04, -1.00000000e+00, -4.71000000e-01,
         -4.18900000e-01],
        [ 1.24447735e+04, -1.10000000e+00, -5.88028000e-01,
         -6.36410707e-01],
        [-8.56909008e-01, -8.96197119e-01,  1.21952104e+04,
         -9.25426077e-01],
        [-1.13893916e+00, -2.10000000e+00,  1.14126084e+04,
         -1.14091406e+00],
        [ 1.20805440e+04, -2.00000000e+00, -1.42422857e+00,
         -1.43620973e+00],
        [ 1.21557810e+04, -2.00000000e+00, -1.65304799e+00,
         -1.64451117e+00],
        [-1.78410196e+00, -2.00000000e+00,  1.16733104e+04,
         -2.00000000e+00]],

       [[ 1.07447993e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.08608230e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.11845547e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.16541957e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e-01],
        [ 1.17390338e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [-2.93835746e-01, -1.00000000e+00,  1.03531996e+04,
         -1.00000000e+00],
        [ 1.09098650e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.04876110e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.04699986e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-1.00000000e+00,  9.99176054e+03, -1.00000000e+00,
         -2.19100000e-01],
        [-1.00000000e+00, -3.90990000e-01, -3.82000000e-01,
          1.00706722e+04],
        [-1.00000000e+00,  1.11158183e+04, -4.58330000e-01,
         -5.30865333e-01],
        [-1.00000000e+00, -6.88347097e-01,  1.07774241e+04,
         -7.89196128e-01],
        [-2.00000000e+00, -1.06965869e+00,  9.88134858e+03,
         -1.07777302e+00],
        [-2.00000000e+00,  9.84382905e+03, -1.27559280e+00,
         -1.27594819e+00],
        [-2.00000000e+00,  9.31313396e+03, -1.39924188e+00,
         -1.40281532e+00],
        [-2.00000000e+00,  1.00939847e+04, -1.51056595e+00,
         -1.51549138e+00],
        [-2.00000000e+00, -1.57176540e+00,  9.03442010e+03,
         -2.00000000e+00]],

       [[-1.90000000e-01,  1.09911679e+04, -1.00000000e+00,
         -1.10000000e-01],
        [-2.81000000e-01, -2.90000000e-01,  1.02022724e+04,
         -2.19000000e-01],
        [-3.81100000e-01,  1.12891860e+04, -3.89100000e-01,
         -4.18794000e-01],
        [-5.85737340e-01,  1.02148696e+04, -5.76806170e-01,
         -6.43132021e-01],
        [-9.09424913e-01,  1.10813407e+04, -8.84198568e-01,
         -9.62287452e-01],
        [-1.12495062e+00, -1.08804852e+00,  1.07696027e+04,
         -1.06243741e+00],
        [-1.24926833e+00, -1.22618026e+00,  1.02070352e+04,
         -1.23341091e+00],
        [-1.41574926e+00,  1.06750919e+04, -1.45223171e+00,
         -1.48114230e+00],
        [-1.58962492e+00,  9.04778321e+03, -1.59482797e+00,
         -2.00000000e+00]],

       [[-1.90000000e-01,  1.14102233e+04, -1.00000000e+00,
         -1.00000000e-01],
        [-2.00000000e-01, -1.90000000e-01,  1.13889658e+04,
         -1.00000000e-01],
        [-3.09000000e-01, -1.00000000e+00,  1.13759236e+04,
         -2.01000000e-01],
        [-4.17280000e-01, -4.48631720e-01,  1.13537044e+04,
         -4.58365100e-01],
        [-7.12371390e-01, -1.00000000e+00,  1.13199786e+04,
         -7.26936401e-01],
        [-9.42330167e-01, -9.98403526e-01,  1.12647099e+04,
         -9.70252550e-01],
        [-1.12458721e+00, -2.00000000e+00,  1.11981888e+04,
         -1.12682267e+00],
        [-1.40325726e+00, -1.41201728e+00,  1.11290961e+04,
         -1.36756537e+00],
        [-1.58391401e+00, -1.60007582e+00,  1.05802585e+04,
         -2.00000000e+00]],

       [[-1.00000000e-01,  1.14640023e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.05379777e+04,
          0.00000000e+00],
        [ 1.02314430e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.11321868e+04, -5.18288212e-01, -1.00000000e+00,
         -1.10000000e+00],
        [ 1.02246815e+04, -1.00000000e+00, -1.20000000e-01,
         -2.04002640e-01],
        [ 1.09582241e+04, -1.15034475e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00, -1.97973916e-01,
          1.00370547e+04],
        [ 1.09943280e+04, -1.63198614e+00, -2.00000000e+00,
         -1.55321188e+00],
        [-1.65897898e+00, -1.61669085e+00,  1.06456184e+04,
         -2.00000000e+00]],

       [[ 1.05853032e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.10143305e+04,
          0.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  8.95267327e+03,
          0.00000000e+00],
        [ 1.07406002e+04, -7.58984920e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.00383035e+04,
          0.00000000e+00],
        [ 1.04570008e+04, -1.31324548e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.68094057e+03,
         -4.00209436e-01],
        [ 9.98468463e+03, -1.78926138e+00, -2.00000000e+00,
         -1.82796562e+00],
        [ 9.60954854e+03, -1.84847645e+00, -1.83516636e+00,
         -2.00000000e+00]],

       [[ 1.15003493e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.14621092e+04,
          0.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.03059138e+04,
          0.00000000e+00],
        [-7.40446561e-01,  9.98474529e+03, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.59343825e-01,
          9.79412376e+03],
        [-1.30956538e+00,  1.08183989e+04, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.71008379e+03,
         -2.68528521e-01],
        [-1.88230360e+00,  9.30604948e+03, -2.00000000e+00,
         -1.89700779e+00],
        [-1.81948414e+00,  8.93149360e+03, -1.84349834e+00,
         -2.00000000e+00]],

       [[-1.00000000e-01, -1.10000000e-01, -1.00000000e+00,
          9.95436244e+03],
        [ 1.14316858e+04, -1.00000000e-01, -1.00000000e-01,
         -1.00000000e-01],
        [-1.00000000e+00, -1.34586156e-01,  1.06497793e+04,
         -1.10000000e-01],
        [-7.93606562e-01,  1.10438180e+04, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00,  1.01869346e+04, -2.83419010e-01,
         -3.75386871e-01],
        [-1.19578425e+00,  1.11175128e+04, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00,  1.02624825e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.64608119e+00,  1.04859693e+04, -2.00000000e+00,
         -1.58531614e+00],
        [-1.75744441e+00, -1.67704166e+00,  9.89479711e+03,
         -2.00000000e+00]],

       [[-2.71000000e-01, -3.01271000e-01, -1.00000000e+00,
          1.02827735e+04],
        [ 1.14062639e+04, -2.00000000e-01, -2.00000000e-01,
         -2.10000000e-01],
        [-1.00000000e+00, -4.00990000e-01,  1.13833515e+04,
         -4.43786000e-01],
        [-6.57895900e-01, -6.88302500e-01,  1.13620540e+04,
         -7.06605070e-01],
        [-1.00000000e+00, -8.91040473e-01,  1.13272525e+04,
         -8.42166605e-01],
        [-1.02746844e+00, -1.04920620e+00,  1.12804869e+04,
         -9.94182335e-01],
        [-2.00000000e+00, -1.18747563e+00,  1.11557193e+04,
         -1.14826871e+00],
        [-1.37730636e+00, -1.38553249e+00,  1.09883586e+04,
         -1.39231002e+00],
        [-1.50072149e+00, -1.54756148e+00,  1.06512144e+04,
         -2.00000000e+00]],

       [[-3.00000000e-01, -1.10000000e+00, -1.00000000e+00,
          1.09041707e+04],
        [ 1.13605200e+04, -1.00000000e+00, -2.90000000e-01,
         -3.44390000e-01],
        [-4.93789000e-01, -1.10000000e+00,  1.12456983e+04,
         -5.36455950e-01],
        [-7.82001727e-01, -1.20000000e+00,  1.09485137e+04,
         -8.09960925e-01],
        [-9.23495740e-01, -1.00021618e+00,  1.04514311e+04,
         -1.01227977e+00],
        [ 1.11543681e+04, -2.00000000e+00, -1.08309109e+00,
         -1.08056791e+00],
        [-1.19088088e+00, -2.00000000e+00,  1.09318969e+04,
         -1.14311854e+00],
        [-1.33992131e+00, -2.10000000e+00,  1.04197794e+04,
         -1.33562160e+00],
        [ 1.01327202e+04, -2.00000000e+00, -1.52770789e+00,
         -2.00000000e+00]],

       [[ 9.81776345e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.09659931e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.03478656e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.02472773e+04, -1.00000000e+00, -1.00000000e+00,
         -6.88546142e-01],
        [ 9.46221737e+03, -2.00000000e+00, -1.21900000e+00,
         -2.00000000e+00],
        [ 1.02765614e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.00595921e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.42948714e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.27873916e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-1.00000000e+00, -2.90000000e-01, -1.00000000e+00,
          9.55326874e+03],
        [-1.00000000e+00, -2.90000000e-01, -3.00000000e-01,
          1.06526773e+04],
        [-1.00000000e+00,  1.11782397e+04, -3.00000000e-01,
         -3.26124238e-01],
        [-1.00000000e+00,  1.00921663e+04, -3.90000000e-01,
         -4.10099000e-01],
        [-1.00000000e+00,  1.11610157e+04, -5.20951000e-01,
         -6.16679907e-01],
        [-1.00000000e+00, -8.05195736e-01,  1.08487462e+04,
         -8.43947512e-01],
        [-1.00000000e+00, -1.03089490e+00,  1.03255842e+04,
         -1.04659930e+00],
        [-2.00000000e+00, -1.21305026e+00,  9.42624859e+03,
         -1.17493965e+00],
        [-2.00000000e+00,  9.02400726e+03, -1.20548968e+00,
         -2.00000000e+00]],

       [[-2.00000000e-01,  1.03531154e+04, -1.00000000e+00,
         -1.00000000e-01],
        [-2.00000000e-01,  1.15704140e+04, -1.00000000e-01,
         -1.00000000e-01],
        [-1.90000000e-01, -1.90000000e-01,  1.14653942e+04,
         -1.10000000e-01],
        [-2.91000000e-01, -2.90000000e-01,  1.10866705e+04,
         -3.10000000e-01],
        [-4.70840000e-01,  1.14415450e+04, -4.61179595e-01,
         -4.12249000e-01],
        [-6.91493830e-01,  1.05802434e+04, -6.81694530e-01,
         -7.63030183e-01],
        [-9.97367946e-01,  1.10600676e+04, -9.13757572e-01,
         -9.92835647e-01],
        [-1.16143100e+00, -1.09294579e+00,  1.01675470e+04,
         -1.12693913e+00],
        [-1.26124470e+00,  1.04525109e+04, -1.22967108e+00,
         -2.00000000e+00]],

       [[-1.00000000e-01,  1.16764887e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.16528601e+04,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00,  1.16198310e+04,
          0.00000000e+00],
        [-2.00000000e-01, -1.90000000e-01,  1.15964990e+04,
         -2.10000000e-01],
        [-3.81000000e-01, -1.00000000e+00,  1.15827675e+04,
         -3.97290000e-01],
        [-5.97546600e-01, -5.28149000e-01,  1.15685439e+04,
         -5.33421690e-01],
        [-8.04457771e-01, -1.00000000e+00,  1.15272890e+04,
         -7.70404031e-01],
        [-1.07042415e+00, -1.08383278e+00,  1.14481723e+04,
         -9.97860000e-01],
        [-1.16652662e+00, -1.15593614e+00,  1.10493717e+04,
         -2.00000000e+00]],

       [[ 1.06853429e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.08830769e+04,
          0.00000000e+00],
        [ 1.06147233e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.14238537e+04, -1.00000000e-01, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.02619815e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.14090942e+04, -4.48758190e-01, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.04015099e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.13333579e+04, -1.14908226e+00, -2.00000000e+00,
         -1.20314713e+00],
        [-1.23084544e+00, -1.26130284e+00,  1.08576765e+04,
         -2.00000000e+00]],

       [[ 1.16744073e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.16477078e+04,
          0.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.06099538e+04,
          0.00000000e+00],
        [ 1.10653691e+04, -3.37110100e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.00303092e+04,
          0.00000000e+00],
        [ 1.12059975e+04, -6.55872815e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.91959604e+03,
          0.00000000e+00],
        [ 1.11648320e+04, -1.39149908e+00, -2.00000000e+00,
         -1.41781870e+00],
        [ 9.90784548e+03, -1.51574030e+00, -1.47080710e+00,
         -2.00000000e+00]],

       [[-1.00000000e-01, -1.10000000e-01, -1.00000000e+00,
          1.03191499e+04],
        [ 1.16172566e+04, -1.00000000e-01, -1.00000000e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.04114218e+04,
          0.00000000e+00],
        [ 9.74660641e+03, -6.19158172e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.49466542e-01,
          9.50119391e+03],
        [ 1.09041954e+04, -9.19656239e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.68850597e+03,
         -6.25800454e-01],
        [ 1.09061774e+04, -1.68548499e+00, -2.01470468e+00,
         -1.63837899e+00],
        [-1.71395856e+00, -1.78163335e+00,  9.34025120e+03,
         -2.00000000e+00]],

       [[-3.36900000e-01, -3.25290000e-01, -1.00000000e+00,
          1.05728845e+04],
        [ 1.15937628e+04, -3.10180000e-01, -3.00900000e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.50025300e-01,  1.08782609e+04,
          0.00000000e+00],
        [-8.07939333e-01,  1.03897098e+04, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00,  1.04208408e+04, -1.40349000e-01,
         -2.05427771e-01],
        [ 9.96702368e+03, -1.14938773e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00,  9.85042926e+03,  0.00000000e+00,
          0.00000000e+00],
        [ 1.04058022e+04, -1.85429271e+00, -2.00000000e+00,
         -1.83000089e+00],
        [-1.89930438e+00, -1.94871614e+00,  9.26269356e+03,
         -2.00000000e+00]],

       [[-5.41669000e-01, -5.24760900e-01, -1.00000000e+00,
          1.11737541e+04],
        [ 1.15758168e+04, -5.16138000e-01, -5.15642000e-01,
         -5.17172900e-01],
        [-1.00000000e+00, -7.00437410e-01,  1.15539814e+04,
         -7.16224676e-01],
        [-8.97251390e-01, -9.42200914e-01,  1.15162791e+04,
         -9.74763407e-01],
        [-2.00000000e+00, -1.26939366e+00,  1.14445140e+04,
         -1.20202634e+00],
        [-1.27575039e+00, -1.26837584e+00,  1.13547574e+04,
         -1.28412411e+00],
        [-2.00000000e+00, -1.57023309e+00,  1.12462856e+04,
         -1.52221908e+00],
        [-1.86695809e+00, -1.88174397e+00,  1.09046944e+04,
         -1.83986889e+00],
        [-2.01731697e+00, -1.99943473e+00,  9.91035095e+03,
         -2.00000000e+00]],

       [[ 1.05509136e+04, -1.00000000e+00, -1.00000000e+00,
         -5.99686000e-01],
        [ 1.07407842e+04, -1.00000000e+00, -6.09288800e-01,
         -6.05588200e-01],
        [ 1.09204131e+04, -1.10000000e+00, -7.82638937e-01,
         -7.54334134e-01],
        [ 1.12969806e+04, -2.00000000e+00, -1.07912710e+00,
         -1.12285820e+00],
        [-1.42476453e+00, -1.35795215e+00,  1.08347891e+04,
         -1.41487262e+00],
        [ 1.09494651e+04, -2.00000000e+00, -1.46750780e+00,
         -1.55857154e+00],
        [ 1.11105167e+04, -2.00000000e+00, -1.65496415e+00,
         -1.69248029e+00],
        [-2.01928808e+00, -2.00000000e+00,  1.09089589e+04,
         -1.93886705e+00],
        [-2.10819924e+00, -3.00000000e+00,  1.05097410e+04,
         -3.00000000e+00]],

       [[ 8.56074714e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.72295664e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.87969125e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.00398953e+04, -1.00000000e+00, -1.00000000e+00,
         -2.34067467e-01],
        [ 9.92824961e+03, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 1.00569198e+04, -1.00000000e+00, -3.52009514e-01,
         -1.00000000e+00],
        [ 1.03147295e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.61558920e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.79171887e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-1.00000000e+00,  1.10623995e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e+00, -2.00000000e-01,  1.00180099e+04,
         -1.11000000e-01],
        [-1.00000000e+00,  1.07128591e+04, -3.62900000e-01,
         -3.27110000e-01],
        [-1.00000000e+00,  1.08095646e+04, -4.80000000e-01,
         -4.09900000e-01],
        [-1.00000000e+00,  1.04470525e+04, -7.27666960e-01,
         -7.49418798e-01],
        [-1.00000000e+00,  9.79831501e+03, -9.52884165e-01,
         -9.51816360e-01],
        [-2.00000000e+00,  9.31357123e+03, -1.19900239e+00,
         -1.25127685e+00],
        [-2.00000000e+00,  1.05610786e+04, -1.40443744e+00,
         -1.42604186e+00],
        [-2.00000000e+00, -1.64600297e+00,  9.13156095e+03,
         -2.00000000e+00]],

       [[-1.00000000e-01,  1.14909352e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.90000000e-01,  1.16976629e+04, -1.00000000e-01,
         -1.10000000e-01],
        [-2.91000000e-01, -2.81000000e-01,  1.16279555e+04,
         -2.09000000e-01],
        [-3.82990000e-01, -3.71900000e-01,  1.15424043e+04,
         -3.26200000e-01],
        [-5.76229000e-01, -6.01340190e-01,  1.13853954e+04,
         -6.44656416e-01],
        [-8.64878225e-01, -8.53360359e-01,  1.09437611e+04,
         -8.38340561e-01],
        [-1.14586639e+00, -1.10235458e+00,  1.02379521e+04,
         -1.10423978e+00],
        [-1.36319986e+00,  1.12604024e+04, -1.35573893e+00,
         -1.33266001e+00],
        [-1.54307576e+00,  1.02007550e+04, -1.46527545e+00,
         -2.00000000e+00]],

       [[-1.00000000e-01,  1.09155805e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.18270820e+04,
          0.00000000e+00],
        [-2.10000000e-01, -1.00000000e+00,  1.17425742e+04,
         -1.11000000e-01],
        [-3.09900000e-01, -3.53900000e-01,  1.17039888e+04,
         -3.38306100e-01],
        [-6.01886400e-01, -1.00000000e+00,  1.16717433e+04,
         -6.39408102e-01],
        [-8.99874130e-01, -9.72794775e-01,  1.16433395e+04,
         -9.64300959e-01],
        [-1.10926246e+00, -2.00000000e+00,  1.16138275e+04,
         -1.13517520e+00],
        [-1.36392900e+00, -1.39854655e+00,  1.15894453e+04,
         -1.33975632e+00],
        [-1.53315172e+00, -1.53370759e+00,  1.11600508e+04,
         -2.00000000e+00]],

       [[ 1.18322408e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.17421572e+04,
          0.00000000e+00],
        [ 1.06495367e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.15628105e+04, -4.01817100e-01, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.04044669e+04, -1.00000000e+00, -1.10000000e-01,
         -1.86834597e-01],
        [ 1.15530946e+04, -1.18916675e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00,  1.03988134e+04,
          0.00000000e+00],
        [ 1.15535924e+04, -1.61454019e+00, -2.00000000e+00,
         -1.62267146e+00],
        [-1.75772281e+00, -1.69873964e+00,  1.05376941e+04,
         -2.00000000e+00]],

       [[ 1.11035220e+04, -1.00000000e-01, -1.00000000e+00,
         -1.10000000e-01],
        [ 1.16849257e+04, -2.00000000e-01, -2.00000000e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.06619361e+04,
          0.00000000e+00],
        [ 1.12929747e+04, -5.36229458e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.48688913e-01,
          9.89422783e+03],
        [ 1.13767779e+04, -1.42615833e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -3.67279484e-01,
          1.05136268e+04],
        [ 1.14974669e+04, -1.90910905e+00, -2.00000000e+00,
         -1.95830253e+00],
        [-2.01656156e+00, -2.07348541e+00,  1.10571171e+04,
         -2.00000000e+00]],

       [[ 9.95388693e+03, -2.10000000e-01, -1.00000000e+00,
         -2.10000000e-01],
        [ 1.16487776e+04, -2.00000000e-01, -2.00000000e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.08068313e+04,
          0.00000000e+00],
        [ 1.07228228e+04, -8.36889972e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.78305188e-01,
          1.01596261e+04],
        [ 1.10751062e+04, -1.64990251e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.00510122e-01,
          1.06377071e+04],
        [ 1.13731647e+04, -2.16017203e+00, -3.00000000e+00,
         -2.17943918e+00],
        [ 1.03349614e+04, -2.21243759e+00, -2.23639411e+00,
         -3.00000000e+00]],

       [[-3.80000000e-01,  9.90163112e+03, -1.00000000e+00,
         -3.48702466e-01],
        [ 1.16195201e+04, -3.27190000e-01, -3.01000000e-01,
         -1.00000000e+00],
        [-1.00000000e+00,  1.02631776e+04, -1.28832700e-01,
         -1.86139248e-01],
        [ 1.02387665e+04, -1.08611895e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.17560924e-01,  9.32094511e+03,
          0.00000000e+00],
        [ 1.03504563e+04, -1.97269000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -6.74444708e-01, -5.64983507e-01,
          9.90589888e+03],
        [ 1.10333690e+04, -2.33658519e+00, -3.01000000e+00,
         -2.31772018e+00],
        [-2.46516048e+00, -2.50241656e+00,  9.86210206e+03,
         -3.00000000e+00]],

       [[-5.56520000e-01, -6.54231857e-01, -1.00000000e+00,
          1.09988315e+04],
        [ 1.15942385e+04, -6.28414340e-01, -6.56685400e-01,
         -6.37065287e-01],
        [-1.00000000e+00, -9.99336979e-01,  1.15642678e+04,
         -9.37307964e-01],
        [-1.11428100e+00, -1.19587871e+00,  1.14269566e+04,
         -1.17144382e+00],
        [-2.00000000e+00, -1.49547162e+00,  1.11703911e+04,
         -1.56943873e+00],
        [-1.88385291e+00, -1.92593567e+00,  1.06569161e+04,
         -1.87977746e+00],
        [-3.00000000e+00, -2.14201805e+00,  9.52629359e+03,
         -2.20490620e+00],
        [ 1.05176402e+04, -2.48987392e+00, -2.40397083e+00,
         -2.42109576e+00],
        [-2.64385720e+00, -2.62963267e+00,  9.56888220e+03,
         -3.00000000e+00]],

       [[-8.21343412e-01, -1.10000000e+00, -1.00000000e+00,
          1.08294037e+04],
        [ 1.13850026e+04, -1.00000000e+00, -8.40371934e-01,
         -8.18225020e-01],
        [ 1.15114340e+04, -1.00000000e+00, -9.99170211e-01,
         -1.02684412e+00],
        [-1.18973255e+00, -2.00000000e+00,  1.14404055e+04,
         -1.19001616e+00],
        [-1.57139569e+00, -1.57386367e+00,  1.13406481e+04,
         -1.68922737e+00],
        [-2.05636801e+00, -3.00000000e+00,  1.11734048e+04,
         -2.12887255e+00],
        [-2.28445273e+00, -3.10000000e+00,  1.09802110e+04,
         -2.32709146e+00],
        [-2.55950342e+00, -3.00000000e+00,  1.07369320e+04,
         -2.59740875e+00],
        [-2.75960240e+00, -3.10000000e+00,  1.03174548e+04,
         -3.00000000e+00]],

       [[ 9.78427646e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.02257255e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.03609098e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.02799040e+04, -1.00000000e+00, -1.00000000e+00,
         -2.39725393e-01],
        [ 1.02303031e+04, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 1.03973832e+04, -1.00000000e+00, -2.57188002e-01,
         -1.00000000e+00],
        [ 1.00269233e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.00105401e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.48118457e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-1.00000000e+00,  1.12735853e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e+00, -1.00000000e-01,  1.14468865e+04,
          0.00000000e+00],
        [-1.00000000e+00,  1.15142408e+04, -1.00000000e-01,
         -1.20000000e-01],
        [-1.00000000e+00,  1.16841082e+04, -2.71000000e-01,
         -2.29800000e-01],
        [-1.00000000e+00,  1.15643935e+04, -4.46600000e-01,
         -4.53282000e-01],
        [-1.00000000e+00,  1.14931310e+04, -7.14937900e-01,
         -7.99950944e-01],
        [-1.00000000e+00,  1.13576295e+04, -9.87014072e-01,
         -9.97979545e-01],
        [-1.00000000e+00,  1.18660515e+04, -1.00538887e+00,
         -1.02646324e+00],
        [-2.00000000e+00, -1.10944386e+00,  1.11046618e+04,
         -2.00000000e+00]],

       [[ 1.22735853e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.22725851e+04,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.22715614e+04,
          0.00000000e+00],
        [-2.00000000e-01, -2.00000000e-01,  1.22702972e+04,
         -1.11000000e-01],
        [-3.81000000e-01, -3.97200000e-01,  1.22679165e+04,
         -4.44452000e-01],
        [-6.84678300e-01, -6.99320590e-01,  1.22632880e+04,
         -6.53520292e-01],
        [-9.30408311e-01, -8.70047835e-01,  1.22239542e+04,
         -8.54014068e-01],
        [-9.69976679e-01, -9.41410324e-01,  1.21737871e+04,
         -9.78308205e-01],
        [-1.07877321e+00, -1.10028846e+00,  1.13952133e+04,
         -2.00000000e+00]],

       [[-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          1.19255022e+04],
        [ 1.22703144e+04, -1.00000000e-01, -1.00000000e-01,
         -1.00000000e-01],
        [ 1.20918132e+04, -1.00000000e+00, -1.00000000e-01,
         -1.10000000e-01],
        [-2.64270000e-01, -2.81000000e-01,  1.19486018e+04,
         -2.20800000e-01],
        [ 1.11566783e+04, -1.00000000e+00, -4.54710000e-01,
         -4.29124120e-01],
        [ 1.22450237e+04, -7.20966545e-01, -7.23827086e-01,
         -7.61815821e-01],
        [-9.16246611e-01, -1.00000000e+00,  1.13644513e+04,
         -9.55146236e-01],
        [ 1.21070017e+04, -1.00944189e+00, -1.04989101e+00,
         -1.02276402e+00],
        [-1.11393126e+00, -1.08518652e+00,  1.14922562e+04,
         -2.00000000e+00]],

       [[ 1.06534188e+04, -2.11200900e-01, -1.00000000e+00,
         -2.19000000e-01],
        [ 1.22620897e+04, -2.10000000e-01, -2.00000000e-01,
         -1.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00,  1.13071459e+04,
          0.00000000e+00],
        [ 1.18232251e+04, -4.00119100e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.53038078e-02, -1.00000000e+00,  1.10744841e+04,
          0.00000000e+00],
        [ 1.22125693e+04, -1.01883868e+00, -1.00000000e+00,
         -1.00000000e+00],
        [-2.75040405e-01, -1.00000000e+00,  1.14826226e+04,
         -1.99377817e-01],
        [ 1.20446345e+04, -1.15634727e+00, -2.01000000e+00,
         -1.09685033e+00],
        [-1.26929267e+00, -1.28292659e+00,  1.14257816e+04,
         -2.00000000e+00]],

       [[-4.46510000e-01, -4.56052864e-01, -1.00000000e+00,
          1.14793739e+04],
        [ 1.22398505e+04, -4.47807854e-01, -3.97200000e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.14940954e+04,
          0.00000000e+00],
        [ 1.15503278e+04, -7.56980706e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.53798800e-01,
          1.13431065e+04],
        [ 1.21655829e+04, -1.04054124e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -3.52422052e-01,
          1.10463049e+04],
        [ 1.19858246e+04, -1.36372325e+00, -2.00000000e+00,
         -1.40392802e+00],
        [-1.47983382e+00, -1.51081679e+00,  1.13588792e+04,
         -2.00000000e+00]],

       [[-6.89062605e-01, -6.91549041e-01, -1.00000000e+00,
          1.16936991e+04],
        [ 1.22051954e+04, -6.91508680e-01, -6.92635054e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.14169958e+04,
          0.00000000e+00],
        [ 1.09326234e+04, -1.00823333e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.53143036e-01,
          1.15175378e+04],
        [ 1.21115377e+04, -1.21356386e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.00000000e-01,
          1.11282704e+04],
        [ 1.19038611e+04, -1.67837017e+00, -2.00000000e+00,
         -1.59873608e+00],
        [-1.72939753e+00, -1.71961527e+00,  1.11651744e+04,
         -2.00000000e+00]],

       [[-9.30506122e-01, -1.01859911e+00, -1.00000000e+00,
          1.11930851e+04],
        [ 1.21589021e+04, -9.37659416e-01, -9.12292111e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -2.01774214e-01,  1.15637534e+04,
          0.00000000e+00],
        [-1.37432097e+00,  1.14397230e+04, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -3.78178117e-01, -4.15813290e-01,
          1.12541260e+04],
        [ 1.20488398e+04, -1.61601846e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.89148921e-01, -1.74569684e-01,
          1.07594937e+04],
        [ 1.16359408e+04, -1.92267819e+00, -2.00000000e+00,
         -1.91140826e+00],
        [-1.99295732e+00, -2.06515450e+00,  1.06954225e+04,
         -2.00000000e+00]],

       [[-1.16844752e+00, -1.22775199e+00, -2.00000000e+00,
          1.09564415e+04],
        [ 1.21061627e+04, -1.19177531e+00, -1.25276734e+00,
         -1.18725929e+00],
        [-2.00000000e+00, -1.41582520e+00,  1.20271990e+04,
         -1.49410105e+00],
        [-1.65001894e+00, -1.66437706e+00,  1.19519023e+04,
         -1.61069890e+00],
        [-2.00000000e+00, -1.89978532e+00,  1.06698375e+04,
         -1.87119635e+00],
        [ 1.19941131e+04, -1.86636192e+00, -1.90166227e+00,
         -1.89120002e+00],
        [-3.00000000e+00, -2.16378061e+00,  1.19298658e+04,
         -2.13044059e+00],
        [-2.14223756e+00, -2.14458006e+00,  1.17052920e+04,
         -2.17141657e+00],
        [-2.27357721e+00, -2.24410233e+00,  1.08708014e+04,
         -3.00000000e+00]],

       [[-1.26108869e+00, -2.00000000e+00, -2.00000000e+00,
          1.16889937e+04],
        [ 1.19907921e+04, -2.00000000e+00, -1.29669613e+00,
         -1.28959637e+00],
        [-1.49961674e+00, -2.00000000e+00,  1.17416823e+04,
         -1.55321120e+00],
        [ 1.18228571e+04, -2.00000000e+00, -1.85422228e+00,
         -1.80596706e+00],
        [-2.06150297e+00, -2.03430790e+00,  1.14645698e+04,
         -2.05176034e+00],
        [ 1.17012841e+04, -2.00000000e+00, -2.02200892e+00,
         -2.04950300e+00],
        [ 1.18456948e+04, -3.00000000e+00, -2.21028217e+00,
         -2.21728340e+00],
        [-2.24480908e+00, -3.00000000e+00,  1.16621087e+04,
         -2.21887608e+00],
        [-2.37596025e+00, -3.00000000e+00,  1.12665172e+04,
         -3.00000000e+00]],

       [[ 1.07780557e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.09809602e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.09828831e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.09597319e+04, -1.00000000e+00, -1.00000000e+00,
         -8.05867380e-01],
        [ 1.05540554e+04, -3.00000000e+00, -2.87519868e+00,
         -3.00000000e+00],
        [ 1.08788367e+04, -1.00000000e+00, -2.99336578e-01,
         -1.00000000e+00],
        [ 1.12376816e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.06136584e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.00873090e+04, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00]]]), array([[[-1.00000000e+00, -1.00000000e-01, -1.00000000e+00,
          9.94927528e+03],
        [-1.00000000e+00,  9.72571835e+03,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e+00, -1.00000000e-01,  1.07256796e+04,
          0.00000000e+00],
        [-1.00000000e+00, -1.00000000e-01,  1.07242871e+04,
          0.00000000e+00],
        [-1.00000000e+00, -2.09000000e-01,  1.07216413e+04,
         -1.00000000e-01],
        [-1.00000000e+00, -2.01000000e-01,  1.06927772e+04,
         -2.19000000e-01],
        [-1.00000000e+00, -4.00000000e-01,  1.06304870e+04,
         -4.15346000e-01],
        [-1.00000000e+00, -6.37497790e-01,  1.05503795e+04,
         -5.89396300e-01],
        [-1.00000000e+00, -7.84524803e-01,  9.61808562e+03,
         -1.00000000e+00]],

       [[-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          9.82170825e+03],
        [ 1.07251412e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.06653540e+04,
          0.00000000e+00],
        [-1.90000000e-01, -1.10000000e-01,  1.05471001e+04,
         -1.10000000e-01],
        [ 1.07069652e+04, -2.90000000e-01, -2.90000000e-01,
         -3.17200000e-01],
        [-3.72900000e-01, -3.91000000e-01,  1.06729169e+04,
         -3.18100000e-01],
        [ 1.00171050e+04, -5.80272910e-01, -5.09125172e-01,
         -5.38854900e-01],
        [ 1.04462126e+04, -7.82909736e-01, -8.02449741e-01,
         -8.23941398e-01],
        [-9.72596126e-01, -1.00184077e+00,  9.54949601e+03,
         -1.00000000e+00]],

       [[-1.90000000e-01, -2.10000000e-01, -1.00000000e+00,
          1.04392457e+04],
        [ 1.07175401e+04, -1.00000000e-01, -1.10000000e-01,
         -1.10000000e-01],
        [ 9.77337104e+03, -1.00000000e+00, -1.00000000e-01,
         -1.10000000e-01],
        [ 1.03961209e+04, -1.90000000e-01, -1.90000000e-01,
         -2.10000000e-01],
        [ 1.02696010e+04, -1.00000000e+00, -3.80000000e-01,
         -3.12931400e-01],
        [ 1.06161166e+04, -4.86878000e-01, -5.62810000e-01,
         -5.54948790e-01],
        [ 9.51419035e+03, -1.00000000e+00, -7.17926861e-01,
         -7.61898577e-01],
        [ 1.03304408e+04, -1.03666598e+00, -9.92778940e-01,
         -1.00886477e+00],
        [-1.14680826e+00, -1.15152809e+00,  9.76585000e+03,
         -2.00000000e+00]],

       [[ 1.00421738e+04, -4.20038600e-01, -1.00000000e+00,
         -3.90100000e-01],
        [ 1.06907695e+04, -3.21610000e-01, -3.16200000e-01,
         -1.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00,  9.91172566e+03,
          0.00000000e+00],
        [ 1.02361788e+04, -2.00000000e-01, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.58249949e+03,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.05348750e+04, -7.75828905e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00,  9.59750039e+03,
          0.00000000e+00],
        [ 1.02045152e+04, -1.39980561e+00, -2.00000000e+00,
         -1.48480730e+00],
        [-1.53293956e+00, -1.62014026e+00,  8.98417515e+03,
         -2.00000000e+00]],

       [[ 9.24618920e+03, -6.42092076e-01, -1.00000000e+00,
         -5.97528400e-01],
        [ 1.06434131e+04, -5.34741720e-01, -5.31361500e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.73043151e+03,
          0.00000000e+00],
        [ 1.00724347e+04, -4.79390843e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.35539975e+03,
          0.00000000e+00],
        [ 1.04580918e+04, -1.23643224e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.41955942e+03,
          0.00000000e+00],
        [ 1.00690313e+04, -1.84932005e+00, -2.00000000e+00,
         -1.81807206e+00],
        [-1.94325401e+00, -1.95900256e+00,  9.81677203e+03,
         -2.00000000e+00]],

       [[-8.18235491e-01, -8.45930265e-01, -1.00000000e+00,
          9.71064608e+03],
        [ 1.05820927e+04, -7.29798911e-01, -7.22556658e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.77649778e+03,
         -1.75200210e-01],
        [ 9.85080335e+03, -8.77785175e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.21432870e+03,
          0.00000000e+00],
        [ 1.03875033e+04, -1.62224726e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.44615359e+03,
         -5.49192301e-01],
        [ 9.66404455e+03, -2.24920057e+00, -3.00000000e+00,
         -2.25756754e+00],
        [ 9.46878399e+03, -2.33924842e+00, -2.27816253e+00,
         -3.00000000e+00]],

       [[-9.40897281e-01, -1.02383851e+00, -1.00000000e+00,
          9.99774827e+03],
        [ 1.05119228e+04, -9.53183456e-01, -9.08393793e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -3.00652098e-01,  9.44299384e+03,
         -1.86318958e-01],
        [ 9.32579947e+03, -1.28715046e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -4.31183993e-01, -3.83361416e-01,
          9.66903385e+03],
        [ 1.03231362e+04, -1.94211511e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00,  9.45311952e+03, -3.71154444e-01,
         -5.32832272e-01],
        [ 8.96720945e+03, -2.54501111e+00, -3.00000000e+00,
         -2.49766502e+00],
        [ 8.66610792e+03, -2.66095527e+00, -2.62057531e+00,
         -3.00000000e+00]],

       [[ 9.19100985e+03, -1.06696083e+00, -2.00000000e+00,
         -1.11704332e+00],
        [ 1.04257518e+04, -1.08399930e+00, -1.10160778e+00,
         -1.05980145e+00],
        [-2.00000000e+00, -1.29489551e+00,  1.03252878e+04,
         -1.32111826e+00],
        [-1.48466913e+00, -1.55151084e+00,  1.02042052e+04,
         -1.54870199e+00],
        [-2.00000000e+00, -1.95936788e+00,  1.00236596e+04,
         -1.95405517e+00],
        [ 1.02595234e+04, -2.13010628e+00, -2.10102244e+00,
         -2.06305745e+00],
        [-3.00000000e+00, -2.34346078e+00,  1.01861284e+04,
         -2.35302551e+00],
        [-2.67857491e+00, -2.66687377e+00,  9.91444644e+03,
         -2.71709468e+00],
        [-2.91723077e+00, -2.95696447e+00,  9.29639109e+03,
         -3.00000000e+00]],

       [[-1.20990398e+00, -2.10000000e+00, -2.00000000e+00,
          9.51018767e+03],
        [ 1.01061178e+04, -2.00000000e+00, -1.24811870e+00,
         -1.19788596e+00],
        [ 9.98970298e+03, -2.00000000e+00, -1.38247741e+00,
         -1.37298742e+00],
        [-1.64301885e+00, -2.00000000e+00,  9.26758041e+03,
         -1.62022383e+00],
        [ 9.74343190e+03, -2.06562974e+00, -1.98433985e+00,
         -2.01021529e+00],
        [ 9.77319644e+03, -2.92336224e+00, -2.26562657e+00,
         -2.26344185e+00],
        [ 1.00576087e+04, -3.00000000e+00, -2.47962493e+00,
         -2.48421483e+00],
        [-2.69296596e+00, -3.00000000e+00,  9.93067739e+03,
         -2.74201930e+00],
        [-2.87372682e+00, -3.00000000e+00,  9.55086722e+03,
         -3.00000000e+00]],

       [[ 8.64011459e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.46223369e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.38324839e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [-1.20900000e-01, -1.00000000e+00, -1.00000000e+00,
          8.71032491e+03],
        [ 9.40303972e+03, -3.00000000e+00, -3.00000000e+00,
         -3.00000000e+00],
        [ 9.10773017e+03, -1.00000000e+00, -5.05037373e-01,
         -1.00000000e+00],
        [ 9.32252266e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.36969067e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 8.76957923e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-1.00000000e+00, -1.10000000e-01, -1.00000000e+00,
          9.91474162e+03],
        [-1.00000000e+00, -1.00000000e-01, -1.00000000e-01,
          1.12234136e+04],
        [-1.00000000e+00,  1.05824873e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e+00,  1.13745759e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e+00, -2.00000000e-01,  1.08045114e+04,
         -1.10000000e-01],
        [-1.00000000e+00, -3.00000000e-01,  9.96080780e+03,
         -3.40030795e-01],
        [-1.00000000e+00,  1.07086766e+04, -4.55610000e-01,
         -4.17317000e-01],
        [-1.00000000e+00,  1.10712658e+04, -6.39524119e-01,
         -7.15134968e-01],
        [-1.00000000e+00, -9.18209068e-01,  9.87017714e+03,
         -1.00000000e+00]],

       [[-1.90000000e-01,  1.03741261e+04, -1.00000000e+00,
         -1.00000000e-01],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e-01,
          1.09256703e+04],
        [ 1.15824873e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.10000000e-01,  1.15814853e+04,
          0.00000000e+00],
        [-1.90000000e-01, -2.00000000e-01,  1.15802364e+04,
         -1.10000000e-01],
        [-2.82100000e-01, -2.99000000e-01,  1.15775367e+04,
         -2.19000000e-01],
        [-4.68040000e-01, -4.09100000e-01,  1.15666317e+04,
         -4.12511000e-01],
        [-6.82543400e-01, -6.33758020e-01,  1.14628842e+04,
         -6.43995312e-01],
        [-8.79016279e-01, -8.42836347e-01,  1.00884821e+04,
         -1.00000000e+00]],

       [[-1.90000000e-01, -2.11100000e-01, -1.00000000e+00,
          1.12948400e+04],
        [-1.90000000e-01, -2.00000000e-01, -2.00000000e-01,
          1.15712288e+04],
        [ 1.15799989e+04, -1.00000000e+00, -1.10000000e-01,
         -1.00000000e-01],
        [ 1.15742049e+04, -1.90000000e-01, -2.00000000e-01,
         -2.10900000e-01],
        [ 1.11564136e+04, -1.00000000e+00, -3.07100000e-01,
         -3.10980100e-01],
        [ 1.15496795e+04, -3.91710000e-01, -3.89910000e-01,
         -3.27100000e-01],
        [ 1.15343906e+04, -1.00000000e+00, -4.80100000e-01,
         -5.22684500e-01],
        [-7.47551220e-01, -7.19667412e-01,  1.14975726e+04,
         -8.24607508e-01],
        [-9.43821656e-01, -9.46636185e-01,  1.05066935e+04,
         -1.00000000e+00]],

       [[-3.63900000e-01, -4.41976900e-01, -1.00000000e+00,
          1.03561146e+04],
        [ 1.15485715e+04, -3.10000000e-01, -2.99000000e-01,
         -1.00000000e+00],
        [ 1.04990477e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.15518586e+04, -2.23828944e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.10000000e-01, -1.00000000e+00,  1.00612709e+04,
          0.00000000e+00],
        [ 1.15133611e+04, -5.91819427e-01, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.02309970e+04, -1.00000000e+00, -1.10000000e-01,
         -1.83217235e-01],
        [ 1.14603314e+04, -9.62702468e-01, -1.00000000e+00,
         -1.01351978e+00],
        [-1.12082145e+00, -1.08653575e+00,  1.02840250e+04,
         -2.00000000e+00]],

       [[-6.11922100e-01, -6.56953325e-01, -1.00000000e+00,
          1.11306398e+04],
        [ 1.15160746e+04, -5.56631818e-01, -5.03569410e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.06080213e+04,
          0.00000000e+00],
        [ 1.15187860e+04, -4.52830716e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.05232193e+04,
          0.00000000e+00],
        [ 1.14813453e+04, -8.79288729e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.69524231e-01,
          1.02573085e+04],
        [ 1.14008774e+04, -1.14364884e+00, -2.00000000e+00,
         -1.22441819e+00],
        [-1.35143163e+00, -1.36196299e+00,  1.07942162e+04,
         -2.00000000e+00]],

       [[ 1.00637695e+04, -9.05494727e-01, -1.00000000e+00,
         -8.15038985e-01],
        [ 1.14779130e+04, -7.84867579e-01, -7.93432117e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.06588804e+04,
          0.00000000e+00],
        [ 1.14710010e+04, -8.48156902e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.06911023e+04,
          0.00000000e+00],
        [ 1.14467411e+04, -1.07087415e+00, -2.00000000e+00,
         -1.02012456e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.01434456e+04,
         -4.05996518e-01],
        [ 1.12575591e+04, -1.57513820e+00, -2.00000000e+00,
         -1.54002936e+00],
        [ 9.53338041e+03, -1.56313009e+00, -1.57189023e+00,
         -2.00000000e+00]],

       [[-1.06331576e+00, -1.08030141e+00, -2.00000000e+00,
          1.08456647e+04],
        [ 1.14222746e+04, -9.82667398e-01, -1.00257236e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -2.21235342e-01,  1.05083806e+04,
          0.00000000e+00],
        [ 1.14044235e+04, -1.12642690e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.20000000e-01,  1.01022234e+04,
         -1.41093322e-01],
        [ 1.13863537e+04, -1.27764847e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00,  9.88771219e+03,  0.00000000e+00,
          0.00000000e+00],
        [ 1.10152486e+04, -1.86010539e+00, -2.00000000e+00,
         -1.83827406e+00],
        [-1.83341073e+00, -1.89129906e+00,  1.06878320e+04,
         -2.00000000e+00]],

       [[ 9.89575378e+03, -1.32861464e+00, -2.00000000e+00,
         -1.26114319e+00],
        [ 1.12889797e+04, -1.25961669e+00, -1.25024537e+00,
         -1.24842600e+00],
        [-2.00000000e+00, -1.38420122e+00,  9.96581086e+03,
         -1.42100408e+00],
        [ 1.13028718e+04, -1.42923558e+00, -1.40332024e+00,
         -1.33873297e+00],
        [-2.00000000e+00, -1.62762529e+00,  1.01832328e+04,
         -1.54271536e+00],
        [ 1.12913434e+04, -1.66197822e+00, -1.64863880e+00,
         -1.64008018e+00],
        [-2.00000000e+00, -1.94161183e+00,  1.11534300e+04,
         -1.96618546e+00],
        [-2.08185636e+00, -2.07522050e+00,  1.07924336e+04,
         -2.13997303e+00],
        [ 1.03117632e+04, -2.09946293e+00, -2.10046477e+00,
         -3.00000000e+00]],

       [[-1.35704756e+00, -2.00000000e+00, -2.00000000e+00,
          1.05548450e+04],
        [ 1.10411987e+04, -2.00000000e+00, -1.40367429e+00,
         -1.43649272e+00],
        [-1.48264895e+00, -2.00000000e+00, -1.43874132e+00,
          1.00254857e+04],
        [ 1.11270110e+04, -2.00000000e+00, -1.58037373e+00,
         -1.58236758e+00],
        [-1.74071568e+00, -1.77638544e+00,  1.07825746e+04,
         -1.82426399e+00],
        [-1.87977737e+00, -2.00000000e+00,  1.02334938e+04,
         -1.94960174e+00],
        [ 1.05669426e+04, -3.00000000e+00, -2.09341968e+00,
         -2.12012243e+00],
        [ 1.02154918e+04, -3.00000000e+00, -2.32453355e+00,
         -2.29597146e+00],
        [ 9.68616810e+03, -2.98330645e+00, -2.22806867e+00,
         -3.00000000e+00]],

       [[ 9.52905134e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.89514551e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 7.98416696e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.58542957e+03, -1.00000000e+00, -1.00000000e+00,
         -2.56370277e-01],
        [ 9.50658330e+03, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 9.26893232e+03, -1.00000000e+00, -2.63072775e-01,
         -1.00000000e+00],
        [ 9.49264306e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.15238705e+03, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 8.68153212e+03, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00]]]), array([[[-1.00000000e+00,  1.10137881e+04, -1.00000000e+00,
         -2.91000000e-01],
        [-1.00000000e+00,  1.18944293e+04, -2.00000000e-01,
         -1.90000000e-01],
        [-1.00000000e+00, -1.00000000e-01, -1.10000000e-01,
          1.17630414e+04],
        [-1.00000000e+00,  1.19683631e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e+00, -1.00000000e-01,  1.28299887e+04,
          0.00000000e+00],
        [-1.00000000e+00, -2.00000000e-01,  1.23300540e+04,
         -1.10000000e-01],
        [-1.00000000e+00, -3.00000000e-01,  1.10848205e+04,
         -3.23017257e-01],
        [-1.00000000e+00,  1.18925580e+04, -3.90000000e-01,
         -3.84872000e-01],
        [-1.00000000e+00, -5.19132290e-01,  1.03486582e+04,
         -1.00000000e+00]],

       [[-4.39410000e-01, -4.00000000e-01, -1.00000000e+00,
          1.19633607e+04],
        [-2.71000000e-01, -2.18000000e-01, -2.00000000e-01,
          1.25859956e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.11000000e-01,
          1.28583626e+04],
        [ 1.29683631e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.25090742e+04,
          0.00000000e+00],
        [-1.90000000e-01, -1.65891000e-01,  1.13041611e+04,
         -1.00000000e-01],
        [-2.81000000e-01, -3.00000000e-01, -2.90000000e-01,
          1.14538192e+04],
        [-3.75700000e-01,  1.26824882e+04, -3.89100000e-01,
         -3.08200000e-01],
        [-5.65043000e-01, -5.45496003e-01,  1.14301717e+04,
         -1.00000000e+00]],

       [[-4.00000000e-01, -4.00000000e-01, -1.00000000e+00,
          1.17745911e+04],
        [-2.81000000e-01, -3.00000000e-01, -3.10000000e-01,
          1.29469221e+04],
        [-1.00000000e-01, -1.00000000e+00, -1.19000000e-01,
          1.29644523e+04],
        [ 1.29673631e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00,  1.29663207e+04,
          0.00000000e+00],
        [-1.90000000e-01, -2.00000000e-01,  1.29648925e+04,
         -1.00000000e-01],
        [-2.90000000e-01, -1.00000000e+00,  1.29550307e+04,
         -2.10000000e-01],
        [-3.72900000e-01, -4.23103660e-01,  1.29274671e+04,
         -4.09190000e-01],
        [-6.44615890e-01, -6.64012370e-01,  1.12337435e+04,
         -1.00000000e+00]],

       [[-4.99710000e-01, -5.53566300e-01, -1.00000000e+00,
          1.17733216e+04],
        [ 1.28983044e+04, -4.03240000e-01, -4.02800000e-01,
         -1.00000000e+00],
        [ 1.17814933e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.29493574e+04, -1.11000000e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.10000000e-02, -1.00000000e+00,  1.17596065e+04,
          0.00000000e+00],
        [ 1.29441882e+04, -2.00000000e-01, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.10367448e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.28494693e+04, -6.86602081e-01, -1.00000000e+00,
         -7.12901201e-01],
        [-8.75620996e-01, -9.39142249e-01,  1.15837555e+04,
         -1.00000000e+00]],

       [[-7.24974990e-01, -7.37936684e-01, -1.00000000e+00,
          1.19466666e+04],
        [ 1.28100551e+04, -6.38350794e-01, -6.35428438e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.52930910e-01,
          1.20810022e+04],
        [ 1.28818541e+04, -3.73955720e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.17609683e+04,
          0.00000000e+00],
        [ 1.28858398e+04, -4.62017951e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.18454386e+04,
          0.00000000e+00],
        [ 1.27370610e+04, -9.89348164e-01, -1.00000000e+00,
         -1.04272234e+00],
        [-1.10288761e+00, -1.17202802e+00,  1.15887977e+04,
         -2.00000000e+00]],

       [[ 1.07811441e+04, -9.95031482e-01, -1.00000000e+00,
         -9.86151368e-01],
        [ 1.26831125e+04, -8.91298945e-01, -9.19348003e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.81876575e-01,
          1.13514476e+04],
        [ 1.26962945e+04, -7.20932684e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.12275819e+04,
          0.00000000e+00],
        [ 1.27831474e+04, -8.21500392e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.18172983e+04,
         -3.34991543e-01],
        [ 1.26027664e+04, -1.32627033e+00, -2.00000000e+00,
         -1.31873133e+00],
        [-1.35484346e+00, -1.42114445e+00,  1.12946716e+04,
         -2.00000000e+00]],

       [[-1.16996561e+00, -1.12308173e+00, -2.00000000e+00,
          1.18976588e+04],
        [ 1.25218615e+04, -1.17092853e+00, -1.12307791e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -5.32410874e-01, -3.55898273e-01,
          1.11688604e+04],
        [ 1.23553883e+04, -1.17658095e+00, -2.01897461e+00,
         -2.00000000e+00],
        [-1.00000000e+00,  1.13309673e+04,  0.00000000e+00,
          0.00000000e+00],
        [ 1.26455794e+04, -1.16361908e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -2.62122271e-01,  1.16168135e+04,
          0.00000000e+00],
        [ 1.24301975e+04, -1.57205487e+00, -2.00000000e+00,
         -1.54080486e+00],
        [-1.67897217e+00, -1.65381581e+00,  1.17073211e+04,
         -2.00000000e+00]],

       [[-1.33152762e+00, -1.36880218e+00, -2.00000000e+00,
          1.10067763e+04],
        [ 1.23250953e+04, -1.37980365e+00, -1.37781395e+00,
         -1.36360823e+00],
        [-2.00000000e+00, -1.59265826e+00,  1.15701488e+04,
         -1.67527947e+00],
        [ 1.19153072e+04, -1.65081303e+00, -1.60664171e+00,
         -1.65816537e+00],
        [-2.00000000e+00, -1.76144992e+00, -1.76385752e+00,
          1.20202675e+04],
        [ 1.24941070e+04, -1.57887554e+00, -1.60568646e+00,
         -1.57045379e+00],
        [-2.00000000e+00, -1.81375861e+00,  1.13727951e+04,
         -1.82956374e+00],
        [ 1.22110612e+04, -1.83550265e+00, -1.86278923e+00,
         -1.83493519e+00],
        [ 1.03737501e+04, -1.88176918e+00, -1.88752662e+00,
         -2.00000000e+00]],

       [[-1.48979661e+00, -2.00000000e+00, -2.00000000e+00,
          1.16838354e+04],
        [ 1.20614363e+04, -2.00000000e+00, -1.61132221e+00,
         -1.60032209e+00],
        [-1.77717245e+00, -2.00000000e+00,  1.16092732e+04,
         -1.79202262e+00],
        [ 1.13409589e+04, -2.00000000e+00, -1.74536193e+00,
         -1.76872516e+00],
        [-1.82791658e+00, -1.83219909e+00, -1.88316834e+00,
          1.14525122e+04],
        [ 1.22368328e+04, -2.00000000e+00, -1.78732036e+00,
         -1.79797460e+00],
        [-2.00210909e+00, -2.10000000e+00,  1.17594790e+04,
         -2.01637976e+00],
        [ 1.18447166e+04, -2.00000000e+00, -2.08022660e+00,
         -1.99526738e+00],
        [-2.02997506e+00, -2.10000000e+00,  1.11878584e+04,
         -3.00000000e+00]],

       [[ 1.04470652e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.08241948e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.10872272e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.03358912e+04, -1.00000000e+00, -1.00000000e+00,
         -2.57512168e-01],
        [ 1.01979076e+04, -3.00000000e+00, -3.00000000e+00,
         -3.00000000e+00],
        [ 1.15549042e+04, -1.00000000e+00, -3.93135918e-01,
         -1.00000000e+00],
        [ 1.08071707e+04, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 1.09208070e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.01977543e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-1.00000000e+00, -3.00000000e-01, -1.00000000e+00,
          1.20941582e+04],
        [-1.00000000e+00, -2.09000000e-01, -2.19000000e-01,
          1.29761951e+04],
        [-1.00000000e+00,  1.34047806e+04, -1.10000000e-01,
         -1.00000000e-01],
        [-1.00000000e+00, -1.00000000e-01, -1.10000000e-01,
          1.30300456e+04],
        [-1.00000000e+00,  1.30425630e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e+00,  1.37553978e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e+00, -1.10000000e-01,  1.33384361e+04,
          0.00000000e+00],
        [-1.00000000e+00, -1.10000000e-01,  1.27897584e+04,
         -1.00000000e-01],
        [-1.00000000e+00, -1.10000000e-01,  1.13854957e+04,
         -1.00000000e+00]],

       [[-2.81000000e-01,  1.21053358e+04, -1.00000000e+00,
         -2.00000000e-01],
        [-1.90000000e-01, -2.00000000e-01, -2.09000000e-01,
          1.24166241e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.10000000e-01,
          1.37270675e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e-01,
          1.39050289e+04],
        [ 1.40240514e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.39648363e+04,
          0.00000000e+00],
        [-1.90000000e-01, -2.00000000e-01,  1.39216410e+04,
         -1.00000000e-01],
        [-2.73901000e-01, -2.09000000e-01,  1.33864543e+04,
         -2.10000000e-01],
        [-3.53900000e-01, -4.03169100e-01,  1.24205582e+04,
         -1.00000000e+00]],

       [[-2.90000000e-01, -3.11829000e-01, -1.00000000e+00,
          1.34866858e+04],
        [-1.90000000e-01, -2.00000000e-01, -2.00000000e-01,
          1.38752718e+04],
        [-1.00000000e-01, -1.00000000e+00, -1.00000000e-01,
          1.39237059e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e-01,
          1.39452625e+04],
        [ 1.39691071e+04, -1.00000000e-01, -1.00000000e-01,
         -1.00000000e-01],
        [-1.90000000e-01, -2.00000000e-01,  1.39407419e+04,
         -1.00000000e-01],
        [ 1.38312322e+04, -1.00000000e+00, -2.00000000e-01,
         -2.19000000e-01],
        [-4.46610000e-01, -4.10000000e-01,  1.36646057e+04,
         -4.00000000e-01],
        [-5.67012000e-01, -5.27462700e-01,  1.26426958e+04,
         -1.00000000e+00]],

       [[ 1.29246489e+04, -5.47330677e-01, -1.00000000e+00,
         -5.11947010e-01],
        [ 1.37530244e+04, -4.28940790e-01, -4.10710000e-01,
         -1.00000000e+00],
        [ 1.27668019e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.38795153e+04, -2.00000000e-01, -1.00000000e+00,
         -1.10000000e+00],
        [-2.00000000e-01, -1.00000000e+00, -2.00000000e-01,
          1.27662546e+04],
        [ 1.39394045e+04, -2.10000000e-01, -1.11000000e+00,
         -1.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00,  1.27817866e+04,
         -5.80948086e-02],
        [ 1.34252002e+04, -7.31186415e-01, -1.00000000e+00,
         -7.00268830e-01],
        [-7.31186780e-01, -7.52031528e-01,  1.30361714e+04,
         -1.00000000e+00]],

       [[ 1.19609018e+04, -7.51238428e-01, -1.00000000e+00,
         -7.57627154e-01],
        [ 1.35953617e+04, -6.57654846e-01, -6.92379712e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.24035465e+04,
          0.00000000e+00],
        [ 1.36977570e+04, -4.27134779e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.23638505e+04,
          0.00000000e+00],
        [ 1.39368306e+04, -5.19392798e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.24816069e+04,
          0.00000000e+00],
        [ 1.31337855e+04, -9.39810872e-01, -1.00000000e+00,
         -1.00794274e+00],
        [ 1.23877040e+04, -9.57597499e-01, -1.02805034e+00,
         -1.00000000e+00]],

       [[-1.06513441e+00, -1.10008245e+00, -2.00000000e+00,
          1.24294541e+04],
        [ 1.33774430e+04, -9.78764098e-01, -9.82295276e-01,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.26505096e+04,
          0.00000000e+00],
        [ 1.33524104e+04, -6.88396358e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.21881950e+04,
          0.00000000e+00],
        [ 1.39305021e+04, -7.28568034e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -3.00035854e-01,
          1.15406288e+04],
        [ 1.28053916e+04, -1.13406040e+00, -2.01719967e+00,
         -1.20859533e+00],
        [-1.17387412e+00, -1.25698686e+00,  1.17194150e+04,
         -2.00000000e+00]],

       [[-1.34135178e+00, -1.44903017e+00, -2.00000000e+00,
          1.22051739e+04],
        [ 1.31595113e+04, -1.21825305e+00, -1.25921684e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -2.08772250e-01,  1.21338314e+04,
          0.00000000e+00],
        [ 1.23447253e+04, -1.05013745e+00, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -2.00000000e-01, -1.81507758e-01,
          1.29937938e+04],
        [ 1.39154137e+04, -9.06914287e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00,  1.23427544e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.40520673e+00,  1.27239102e+04, -2.00000000e+00,
         -1.35388622e+00],
        [-1.42456752e+00,  1.21383840e+04, -1.37560245e+00,
         -2.00000000e+00]],

       [[-1.59445036e+00, -1.61094518e+00, -2.00000000e+00,
          1.10185867e+04],
        [ 1.25558777e+04, -1.58179078e+00, -1.52388680e+00,
         -1.56042908e+00],
        [-2.00000000e+00, -1.50939281e+00, -1.51682988e+00,
          1.22249370e+04],
        [-1.25434586e+00, -1.27536553e+00, -1.24059448e+00,
          1.35460298e+04],
        [-2.00000000e+00, -1.19735009e+00, -1.21026279e+00,
          1.37412243e+04],
        [ 1.38837831e+04, -1.14003949e+00, -1.07594768e+00,
         -1.11956255e+00],
        [-2.00000000e+00, -1.31379271e+00,  1.37815678e+04,
         -1.35230339e+00],
        [-1.42694397e+00, -1.42043251e+00,  1.36120412e+04,
         -1.42205017e+00],
        [-1.44532420e+00, -1.43843735e+00,  1.33441840e+04,
         -2.00000000e+00]],

       [[-1.92946823e+00, -2.00000000e+00, -2.00000000e+00,
          1.23422663e+04],
        [-1.85338903e+00, -2.00000000e+00, -1.87789373e+00,
          1.28674347e+04],
        [-1.67176880e+00, -2.00000000e+00, -1.66419222e+00,
          1.31282192e+04],
        [ 1.33495378e+04, -2.00000000e+00, -1.44766986e+00,
         -1.41588836e+00],
        [ 1.31596446e+04, -1.35961213e+00, -1.39416867e+00,
         -1.38694435e+00],
        [ 1.32210399e+04, -2.00000000e+00, -1.22478666e+00,
         -1.28111022e+00],
        [ 1.31771619e+04, -2.00000000e+00, -1.41019564e+00,
         -1.48104248e+00],
        [ 1.31050945e+04, -2.00000000e+00, -1.51825292e+00,
         -1.49738690e+00],
        [ 1.29190621e+04, -2.00000000e+00, -1.49108242e+00,
         -2.00000000e+00]],

       [[ 1.12267585e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.14829683e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.20649813e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.21186551e+04, -1.00000000e+00, -1.00000000e+00,
         -4.53736965e-01],
        [ 1.20612754e+04, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 1.21081270e+04, -1.00000000e+00, -2.43653467e-01,
         -1.00000000e+00],
        [ 1.21280271e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.20261914e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.18253404e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-1.00000000e+00,  9.78710453e+03, -1.00000000e+00,
         -5.04435100e-01],
        [-1.00000000e+00, -3.09900000e-01, -3.85872000e-01,
          1.05653705e+04],
        [-1.00000000e+00,  1.11099042e+04, -2.00000000e-01,
         -2.00000000e-01],
        [-1.00000000e+00, -2.00000000e-01,  1.06141157e+04,
         -1.90000000e-01],
        [-1.00000000e+00, -1.00000000e-01, -1.10000000e-01,
          1.10929519e+04],
        [-1.00000000e+00,  1.06673912e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e+00,  1.10309467e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e+00, -1.10000000e-01,  1.05084164e+04,
          0.00000000e+00],
        [-1.00000000e+00, -1.10000000e-01,  9.59078093e+03,
         -1.00000000e+00]],

       [[-4.58500000e-01, -5.19320300e-01, -1.00000000e+00,
          1.07087601e+04],
        [-3.71900000e-01, -4.08190000e-01, -3.97200000e-01,
          1.11074521e+04],
        [-2.81000000e-01, -2.81000000e-01, -3.00000000e-01,
          1.13721824e+04],
        [-1.90000000e-01, -2.00000000e-01, -2.09000000e-01,
          1.15331017e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.10000000e-01,
          1.16638217e+04],
        [ 1.16673716e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.13625871e+04,
          0.00000000e+00],
        [-1.90000000e-01,  1.06089673e+04, -1.90000000e-01,
         -2.01000000e-01],
        [-2.71000000e-01,  1.02676400e+04, -2.10000000e-01,
         -1.00000000e+00]],

       [[-7.17073555e-01, -7.30967668e-01, -1.00000000e+00,
          1.06591112e+04],
        [-5.52058401e-01, -5.36827265e-01, -6.10754523e-01,
          1.13386983e+04],
        [-3.00000000e-01, -1.00000000e+00, -3.37371910e-01,
          1.14820325e+04],
        [-2.00000000e-01, -1.90000000e-01, -2.01000000e-01,
          1.16202216e+04],
        [ 1.16501737e+04, -1.00000000e+00, -1.00000000e-01,
         -1.10000000e-01],
        [ 1.16617992e+04, -1.10000000e-01, -1.00000000e-01,
         -1.00000000e-01],
        [-1.00000000e-01, -1.00000000e+00,  1.16180626e+04,
          0.00000000e+00],
        [-1.90000000e-01, -2.10000000e-01,  1.15578456e+04,
         -1.00000000e-01],
        [-2.81000000e-01, -2.10900000e-01,  1.09016313e+04,
         -1.00000000e+00]],

       [[-9.58759937e-01, -9.74938659e-01, -1.00000000e+00,
          1.08687510e+04],
        [ 1.11814078e+04, -9.13282304e-01, -8.11848894e-01,
         -1.00000000e+00],
        [-1.29100000e-01, -1.00000000e+00,  1.04856548e+04,
          0.00000000e+00],
        [ 1.15514052e+04, -3.49287809e-01, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.08903481e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.16010517e+04, -3.96410000e-01, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.06261432e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.14847253e+04, -4.60475593e-01, -1.00000000e+00,
         -4.04138691e-01],
        [-5.20949727e-01, -5.83707274e-01,  1.07218357e+04,
         -1.00000000e+00]],

       [[ 1.00654278e+04, -1.17176506e+00, -2.00000000e+00,
         -1.21165362e+00],
        [ 1.10667180e+04, -1.16647023e+00, -1.18450524e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.10000000e-01,
          1.07654290e+04],
        [ 1.14781516e+04, -7.15681256e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.06678092e+04,
          0.00000000e+00],
        [ 1.15059693e+04, -5.75984760e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.05626413e+04,
          0.00000000e+00],
        [ 1.14074409e+04, -6.75287195e-01, -1.00000000e+00,
         -7.01548123e-01],
        [-8.45504776e-01, -9.04866448e-01,  1.07808881e+04,
         -1.00000000e+00]],

       [[-1.52058810e+00, -1.47710732e+00, -2.00000000e+00,
          1.07654246e+04],
        [ 1.09700804e+04, -1.48551834e+00, -1.48239573e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.16604580e-01,
          1.02937448e+04],
        [ 1.13952012e+04, -9.25321906e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.05507320e+04,
          0.00000000e+00],
        [ 1.14076453e+04, -9.09372044e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.07625834e+04,
          0.00000000e+00],
        [ 1.13350840e+04, -1.01300024e+00, -1.00000000e+00,
         -9.02267962e-01],
        [-1.06267956e+00, -1.10877610e+00,  1.07062677e+04,
         -2.00000000e+00]],

       [[ 1.04743085e+04, -1.85740819e+00, -2.00000000e+00,
         -1.76087231e+00],
        [ 1.04071488e+04, -1.76006126e+00, -1.77976349e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -4.36090754e-01, -4.48550532e-01,
          1.08838147e+04],
        [ 1.13028042e+04, -1.15174518e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -3.19491388e-01, -2.03673506e-01,
          1.06847363e+04],
        [ 1.13074617e+04, -1.10159551e+00, -2.01110000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.91670747e-01, -1.88533480e-01,
          1.05846024e+04],
        [ 1.12614151e+04, -1.38286615e+00, -2.00000000e+00,
         -1.36307370e+00],
        [-1.41141643e+00, -1.39768242e+00,  1.05957851e+04,
         -2.00000000e+00]],

       [[ 9.43689994e+03, -1.95527740e+00, -2.00000000e+00,
         -1.94178476e+00],
        [-1.87681173e+00, -1.81402235e+00, -1.83334313e+00,
          1.08411802e+04],
        [-2.00000000e+00, -1.66870850e+00, -1.71390308e+00,
          1.09849159e+04],
        [ 1.11927859e+04, -1.57890143e+00, -1.49308045e+00,
         -1.49567472e+00],
        [-2.00000000e+00, -1.69258547e+00, -1.59323861e+00,
          1.05039849e+04],
        [ 1.12026840e+04, -1.57041554e+00, -1.47522061e+00,
         -1.47926977e+00],
        [-2.00000000e+00, -1.61687205e+00,  1.07805781e+04,
         -1.57393198e+00],
        [ 1.11537633e+04, -1.63107929e+00, -1.56247450e+00,
         -1.62445359e+00],
        [-1.62361185e+00, -1.69698538e+00,  1.09259108e+04,
         -2.00000000e+00]],

       [[-2.13600197e+00, -2.10000000e+00, -3.00000000e+00,
          1.02389055e+04],
        [ 1.06919826e+04, -2.00000000e+00, -1.99032275e+00,
         -2.02769551e+00],
        [-1.82660311e+00, -2.00000000e+00, -1.88468056e+00,
          1.07760737e+04],
        [ 1.10476761e+04, -2.00000000e+00, -1.75872484e+00,
         -1.82237002e+00],
        [-1.78290112e+00, -1.73384185e+00, -1.77505677e+00,
          1.08298947e+04],
        [ 1.10684865e+04, -2.00000000e+00, -1.70281231e+00,
         -1.71176668e+00],
        [-1.77396020e+00, -2.00000000e+00,  1.08179419e+04,
         -1.83613912e+00],
        [ 1.07777265e+04, -2.00000000e+00, -1.85338860e+00,
         -1.89540641e+00],
        [ 1.06283341e+04, -2.00000000e+00, -1.89925583e+00,
         -2.00000000e+00]],

       [[ 9.21973178e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.01017427e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.02260421e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.05911436e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.03161514e+04, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 1.04444384e+04, -1.00000000e+00, -2.75320525e-01,
         -1.00000000e+00],
        [ 1.02512613e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.93389815e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.84027513e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-1.00000000e+00, -8.88337186e-01, -1.00000000e+00,
          9.10155679e+03],
        [-1.00000000e+00, -5.67343720e-01, -6.27050500e-01,
          1.00325244e+04],
        [-1.00000000e+00,  1.04381354e+04, -4.31707300e-01,
         -3.81000000e-01],
        [-1.00000000e+00,  9.99358414e+03, -2.01000000e-01,
         -2.82000000e-01],
        [-1.00000000e+00,  9.34927169e+03, -1.10000000e-01,
         -1.90000000e-01],
        [-1.00000000e+00, -1.00000000e-01, -1.10000000e-01,
          9.71027488e+03],
        [-1.00000000e+00,  9.66039234e+03,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e+00, -1.00000000e-01,  1.05129973e+04,
          0.00000000e+00],
        [-1.00000000e+00, -1.00000000e-01,  9.58124851e+03,
          0.00000000e+00]],

       [[-7.94941353e-01, -7.08983098e-01, -1.00000000e+00,
          1.03754256e+04],
        [-5.64179000e-01, -5.94507220e-01, -5.16650000e-01,
          1.05657112e+04],
        [-3.82900000e-01, -4.00710000e-01, -3.34390000e-01,
          1.06223985e+04],
        [-2.90000000e-01, -2.99000000e-01, -2.09000000e-01,
          1.06518173e+04],
        [-1.90000000e-01, -1.90000000e-01, -2.00000000e-01,
          1.06571972e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.10000000e-01,
          1.06592691e+04],
        [ 1.06603909e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 9.68417129e+03, -1.00000000e-01, -1.00000000e-01,
         -1.00000000e-01],
        [-1.90000000e-01,  9.20442815e+03, -1.00000000e-01,
         -1.00000000e+00]],

       [[ 1.01149278e+04, -9.49995431e-01, -1.00000000e+00,
         -8.73731485e-01],
        [ 1.03736453e+04, -7.48600355e-01, -8.54135504e-01,
         -8.03048738e-01],
        [ 9.57518910e+03, -1.10000000e+00, -5.54627750e-01,
         -4.84851000e-01],
        [ 1.06279366e+04, -3.62900000e-01, -3.01000000e-01,
         -3.63000000e-01],
        [-2.00000000e-01, -1.00000000e+00, -1.10000000e-01,
          1.01608686e+04],
        [ 1.06184474e+04, -1.00000000e-01, -1.00000000e-01,
         -1.00000000e-01],
        [ 1.06518101e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.06297312e+04,
          0.00000000e+00],
        [-1.00000000e-01, -1.10000000e-01,  1.02163363e+04,
          0.00000000e+00]],

       [[ 9.83123188e+03, -1.14485131e+00, -2.00000000e+00,
         -1.08558904e+00],
        [ 1.01078298e+04, -1.15614823e+00, -1.15309792e+00,
         -2.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00, -1.75360649e-01,
          9.63883724e+03],
        [ 1.05986597e+04, -3.86673144e-01, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.38089115e+03,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.05430387e+04, -2.09000000e-01, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.64813559e+03,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.06005306e+04, -1.00000000e-01, -1.00000000e+00,
         -1.00000000e-01],
        [-1.90000000e-01, -1.10000000e-01,  9.75940958e+03,
         -1.00000000e+00]],

       [[ 9.46574182e+03, -1.35853963e+00, -2.00000000e+00,
         -1.39402373e+00],
        [ 9.73861813e+03, -1.44451134e+00, -1.41383018e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.23035909e-01,
          9.78302279e+03],
        [ 1.05745318e+04, -5.78047265e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.71140430e+03,
          0.00000000e+00],
        [ 1.04241045e+04, -3.85589928e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.48128858e+03,
          0.00000000e+00],
        [ 1.05646583e+04, -3.39718120e-01, -1.00000000e+00,
         -3.15200000e-01],
        [-4.08713011e-01, -4.35806827e-01,  9.79700021e+03,
         -1.00000000e+00]],

       [[ 8.86014456e+03, -1.78059871e+00, -2.00000000e+00,
         -1.78126939e+00],
        [ 9.21990807e+03, -1.71331724e+00, -1.75205424e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.61283250e-01,
          9.54737626e+03],
        [ 1.05568969e+04, -8.50608371e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.66800641e+03,
          0.00000000e+00],
        [ 1.01659464e+04, -6.80730366e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.21302800e-01,
          9.62510053e+03],
        [ 1.05312356e+04, -5.55523631e-01, -1.00000000e+00,
         -6.01943080e-01],
        [-6.96150300e-01, -6.68569910e-01,  9.47519892e+03,
         -1.00000000e+00]],

       [[ 7.87147961e+03, -1.94656200e+00, -2.00000000e+00,
         -1.94441197e+00],
        [ 8.10944102e+03, -1.96390771e+00, -1.98629605e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -3.83551110e-01, -4.67438887e-01,
          9.63668964e+03],
        [ 1.05350474e+04, -1.07117755e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -3.35492182e-01,  9.64718356e+03,
         -1.70041022e-01],
        [ 9.83574431e+03, -1.05152795e+00, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -2.01412183e-01,  8.65899433e+03,
          0.00000000e+00],
        [ 1.04558130e+04, -7.72528161e-01, -1.00000000e+00,
         -7.87741486e-01],
        [-9.63776275e-01, -9.70513839e-01,  9.27852172e+03,
         -1.00000000e+00]],

       [[-2.19087209e+00, -2.23431110e+00, -3.00000000e+00,
          9.40450079e+03],
        [-2.12238213e+00, -2.11765026e+00, -2.12976804e+00,
          9.78378144e+03],
        [-2.00000000e+00, -1.95951281e+00, -1.98051346e+00,
          1.01297776e+04],
        [ 1.04840028e+04, -1.50688057e+00, -1.55108104e+00,
         -1.56800272e+00],
        [-2.00000000e+00, -1.59704074e+00,  1.00665798e+04,
         -1.62161109e+00],
        [ 8.75222324e+03, -1.47360644e+00, -1.42139447e+00,
         -1.45590950e+00],
        [-2.00000000e+00, -1.49832856e+00, -1.44063060e+00,
          9.00667865e+03],
        [ 1.03215613e+04, -1.14038065e+00, -1.21902226e+00,
         -1.14487804e+00],
        [-1.10217427e+00, -1.12616950e+00,  9.87995728e+03,
         -2.00000000e+00]],

       [[ 8.93541306e+03, -3.00000000e+00, -3.00000000e+00,
         -2.42801221e+00],
        [-2.19617291e+00, -3.00000000e+00, -2.27222723e+00,
          9.43294060e+03],
        [-2.01739372e+00, -3.00000000e+00, -2.01485115e+00,
          1.00100915e+04],
        [ 1.02832263e+04, -2.00000000e+00, -1.77540717e+00,
         -1.80312311e+00],
        [ 9.67341826e+03, -1.84241299e+00, -1.85872178e+00,
         -1.80325176e+00],
        [-1.74599864e+00, -2.00000000e+00, -1.78523682e+00,
          9.09773758e+03],
        [-1.55118900e+00, -2.00000000e+00, -1.60516772e+00,
          9.72648418e+03],
        [ 1.00362334e+04, -2.10000000e+00, -1.35568765e+00,
         -1.26579991e+00],
        [ 9.17013601e+03, -2.00000000e+00, -1.12433889e+00,
         -2.00000000e+00]],

       [[ 7.95638594e+03, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 8.30777966e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.15082194e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.42494033e+03, -1.00000000e+00, -1.00000000e+00,
         -2.68532178e-01],
        [ 8.73395602e+03, -3.00000000e+00, -3.00000000e+00,
         -3.00000000e+00],
        [ 7.92316369e+03, -1.00000000e+00, -7.33300559e-01,
         -1.00000000e+00],
        [ 8.93885201e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.19828231e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 8.31841348e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-2.00000000e+00,  9.15843557e+03, -2.00000000e+00,
         -1.14907479e+00],
        [-1.00000000e+00,  9.64471170e+03, -1.03564103e+00,
         -9.90215905e-01],
        [-1.00000000e+00, -7.40841817e-01, -7.59718090e-01,
          9.23616795e+03],
        [-1.00000000e+00,  1.03031610e+04, -5.42709871e-01,
         -5.39949000e-01],
        [-1.00000000e+00,  1.02714595e+04, -3.42006029e-01,
         -3.72900000e-01],
        [-1.00000000e+00,  9.55029742e+03, -2.20800000e-01,
         -2.71000000e-01],
        [-1.00000000e+00, -1.00000000e-01, -1.11000000e-01,
          1.01222726e+04],
        [-1.00000000e+00,  9.90709841e+03,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e+00, -1.00000000e-01,  1.00305708e+04,
          0.00000000e+00]],

       [[-1.11619710e+00, -1.15186691e+00, -2.00000000e+00,
          1.03451552e+04],
        [-1.00602740e+00, -9.59968829e-01, -1.02505126e+00,
          1.07477047e+04],
        [-7.60931571e-01, -7.24908128e-01, -7.60117837e-01,
          1.07778934e+04],
        [-4.68220000e-01, -5.02384310e-01, -5.33234290e-01,
          1.08108800e+04],
        [-2.82000000e-01, -2.91000000e-01, -3.08200000e-01,
          1.08350547e+04],
        [-2.00000000e-01, -1.90000000e-01, -1.10000000e-01,
          1.08665652e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e-01,
          1.08931298e+04],
        [ 1.09060220e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  9.48756894e+03,
          0.00000000e+00]],

       [[-1.32008002e+00, -1.31049740e+00, -2.00000000e+00,
          1.01420528e+04],
        [ 1.07059755e+04, -1.12981369e+00, -1.10672216e+00,
         -1.08669072e+00],
        [-8.12209088e-01, -1.00000000e+00, -8.54527554e-01,
          9.40376976e+03],
        [ 1.07837959e+04, -5.93297838e-01, -5.52281650e-01,
         -5.48130000e-01],
        [ 1.00519179e+04, -1.00000000e+00, -3.36720000e-01,
         -3.72000000e-01],
        [ 1.07601614e+04, -2.81000000e-01, -2.29000000e-01,
         -2.00000000e-01],
        [ 1.04107604e+04, -1.00000000e+00, -1.10000000e-01,
         -1.00000000e-01],
        [ 1.08669562e+04, -1.00000000e-01, -1.00000000e-01,
         -1.00000000e-01],
        [-1.90000000e-01, -2.19190000e-01,  9.68424969e+03,
         -1.00000000e+00]],

       [[ 9.17364052e+03, -1.53841217e+00, -2.00000000e+00,
         -1.47069220e+00],
        [ 1.06460958e+04, -1.33064827e+00, -1.36244020e+00,
         -2.00000000e+00],
        [-1.46722000e-01, -1.00000000e+00,  9.61187442e+03,
          0.00000000e+00],
        [ 1.07442619e+04, -7.76767528e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-9.91723639e-02, -1.00000000e+00, -1.70781972e-01,
          9.53903885e+03],
        [ 1.06602619e+04, -4.15486464e-01, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.34341518e+03, -1.00000000e+00, -1.00000000e-01,
         -1.20000000e-01],
        [ 1.08243866e+04, -2.90000000e-01, -1.00000000e+00,
         -3.07100000e-01],
        [-3.62900000e-01, -3.94610000e-01,  9.94168742e+03,
         -1.00000000e+00]],

       [[-1.75924181e+00, -1.79616263e+00, -2.00000000e+00,
          1.02009055e+04],
        [ 1.05408126e+04, -1.62954729e+00, -1.67453621e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.73670039e+03,
          0.00000000e+00],
        [ 1.06960824e+04, -1.07589771e+00, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.91455486e+03,
          0.00000000e+00],
        [ 1.05241953e+04, -6.82424676e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  8.93955941e+03,
          0.00000000e+00],
        [ 1.07901257e+04, -4.01800000e-01, -1.00000000e+00,
         -4.00000000e-01],
        [-4.97200000e-01, -5.04410000e-01,  9.94877633e+03,
         -1.00000000e+00]],

       [[ 9.69100202e+03, -1.95733366e+00, -2.00000000e+00,
         -2.00187789e+00],
        [ 1.02608995e+04, -1.92073128e+00, -1.93719214e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -3.32556370e-01,
          9.81463480e+03],
        [ 1.06443138e+04, -1.30054511e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.75848286e-01,
          9.19720221e+03],
        [ 1.02827417e+04, -9.50808167e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.54183307e-01,
          9.87170738e+03],
        [ 1.07617896e+04, -6.58850758e-01, -1.00000000e+00,
         -5.99986970e-01],
        [-6.52793000e-01, -6.35492441e-01,  1.03345050e+04,
         -1.00000000e+00]],

       [[ 8.93492981e+03, -2.16892301e+00, -3.00000000e+00,
         -2.19308878e+00],
        [ 9.92060617e+03, -2.21020184e+00, -2.12655231e+00,
         -3.00000000e+00],
        [-1.00000000e+00, -2.98537986e-01, -3.01974294e-01,
          9.13288845e+03],
        [ 1.05731632e+04, -1.72229106e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -2.88602173e-01, -1.44482399e-01,
          8.82716486e+03],
        [ 9.77628041e+03, -1.30568659e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.78477680e-01, -1.91550785e-01,
          1.00167620e+04],
        [ 1.07332728e+04, -9.84067785e-01, -1.00000000e+00,
         -9.71683186e-01],
        [ 9.65231932e+03, -9.69049443e-01, -9.45956029e-01,
         -1.00000000e+00]],

       [[-2.44300743e+00, -2.49094584e+00, -3.00000000e+00,
          8.49525665e+03],
        [ 9.39636980e+03, -2.35469221e+00, -2.26977946e+00,
         -2.28799648e+00],
        [-3.00000000e+00, -2.16513932e+00, -2.18052403e+00,
          1.03287125e+04],
        [ 1.04864402e+04, -2.01468993e+00, -2.00442926e+00,
         -2.00993215e+00],
        [-2.00000000e+00, -1.96602533e+00, -1.92491968e+00,
          1.00800079e+04],
        [-1.55181786e+00, -1.60401191e+00, -1.58281385e+00,
          1.04689702e+04],
        [-2.00000000e+00, -1.34883514e+00, -1.28568632e+00,
          1.06438433e+04],
        [ 1.07020969e+04, -1.14539805e+00, -1.10133830e+00,
         -1.13061185e+00],
        [-1.10026973e+00,  9.05524598e+03, -1.06750037e+00,
         -2.00000000e+00]],

       [[-2.61829910e+00, -3.00000000e+00, -3.00000000e+00,
          9.32775891e+03],
        [-2.50252658e+00, -3.00000000e+00, -2.58911000e+00,
          9.92605981e+03],
        [ 1.01730837e+04, -3.00000000e+00, -2.38909529e+00,
         -2.29681527e+00],
        [ 1.00522510e+04, -3.00000000e+00, -2.10512425e+00,
         -2.10112077e+00],
        [ 9.44775328e+03, -2.03819353e+00, -2.06941708e+00,
         -2.01939863e+00],
        [ 9.78451413e+03, -2.00000000e+00, -1.79907261e+00,
         -1.77029265e+00],
        [ 1.02542663e+04, -2.00000000e+00, -1.55443539e+00,
         -1.44963122e+00],
        [ 1.04923551e+04, -2.00000000e+00, -1.17258548e+00,
         -1.17257114e+00],
        [-1.13001554e+00, -2.00000000e+00,  1.02008698e+04,
         -2.00000000e+00]],

       [[ 7.66515243e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.05665366e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 8.97020131e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.00483400e+03, -1.00000000e+00, -1.00000000e+00,
         -5.84842273e-01],
        [ 8.31926940e+03, -3.00000000e+00, -2.91000000e+00,
         -3.06016623e+00],
        [ 8.79940014e+03, -1.00000000e+00, -9.04695080e-01,
         -1.00000000e+00],
        [ 9.50966111e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.42576863e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.46911775e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-2.00000000e+00,  1.14080328e+04, -2.00000000e+00,
         -1.30220708e+00],
        [-2.00000000e+00,  1.16988166e+04, -1.22479500e+00,
         -1.23496777e+00],
        [-2.00000000e+00, -1.05188502e+00, -1.04710010e+00,
          1.10347096e+04],
        [-1.00000000e+00,  1.23762230e+04, -8.45170528e-01,
         -7.41671690e-01],
        [-1.00000000e+00,  1.22328984e+04, -5.58174832e-01,
         -5.56185100e-01],
        [-1.00000000e+00,  1.19911760e+04, -3.28720000e-01,
         -3.08100000e-01],
        [-1.00000000e+00,  1.20541632e+04, -2.00000000e-01,
         -2.71000000e-01],
        [-1.00000000e+00, -1.00000000e-01, -1.19000000e-01,
          1.20405439e+04],
        [-1.00000000e+00,  1.19304888e+04,  0.00000000e+00,
          0.00000000e+00]],

       [[-1.40166028e+00, -1.38918161e+00, -2.00000000e+00,
          1.23197293e+04],
        [-1.23447832e+00, -1.30881088e+00, -1.23222253e+00,
          1.26667336e+04],
        [-1.04779498e+00, -9.72511569e-01, -1.07119581e+00,
          1.27076183e+04],
        [-7.50775010e-01, -7.46860740e-01, -7.05466501e-01,
          1.27260306e+04],
        [-4.69760000e-01, -4.67681000e-01, -4.40141000e-01,
          1.27302776e+04],
        [-2.82000000e-01, -3.00000000e-01, -3.01900000e-01,
          1.27375267e+04],
        [-1.90000000e-01, -1.90000000e-01, -2.00000000e-01,
          1.27586404e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.10000000e-01,
          1.28121335e+04],
        [ 1.28776473e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[-1.62208831e+00, -1.65758917e+00, -2.00000000e+00,
          1.15924072e+04],
        [ 1.25725822e+04, -1.54982241e+00, -1.51817968e+00,
         -1.46676689e+00],
        [ 1.18396948e+04, -2.00000000e+00, -1.10879908e+00,
         -1.02413056e+00],
        [ 1.26919291e+04, -8.36659750e-01, -7.83976608e-01,
         -7.37184100e-01],
        [ 1.22197854e+04, -1.00000000e+00, -5.49834956e-01,
         -5.03621120e-01],
        [-2.99000000e-01, -3.81000000e-01, -3.11990000e-01,
          1.26952332e+04],
        [-2.00000000e-01, -1.00000000e+00, -2.10000000e-01,
          1.27242485e+04],
        [ 1.27481981e+04, -1.00000000e-01, -1.10000000e-01,
         -1.00000000e-01],
        [-1.00000000e-01, -1.00000000e-01,  1.17541542e+04,
          0.00000000e+00]],

       [[-1.88454177e+00, -1.90741779e+00, -2.00000000e+00,
          1.15418825e+04],
        [ 1.23715941e+04, -1.87587681e+00, -1.86736398e+00,
         -2.00000000e+00],
        [-1.98709148e-01, -1.00000000e+00, -2.55567915e-01,
          1.17462788e+04],
        [ 1.26179074e+04, -1.02095051e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.12263381e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.26057161e+04, -4.97038313e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.19000000e-01, -1.00000000e+00,  1.13041721e+04,
          0.00000000e+00],
        [ 1.27324688e+04, -1.00000000e-01, -1.00000000e+00,
         -1.00000000e-01],
        [-1.90000000e-01,  1.12519682e+04, -1.00000000e-01,
         -1.00000000e+00]],

       [[-2.13309769e+00, -2.15995526e+00, -3.00000000e+00,
          1.13993783e+04],
        [ 1.19983186e+04, -2.24112048e+00, -2.13547231e+00,
         -3.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -3.05550957e-01,
          1.15139570e+04],
        [ 1.24475501e+04, -1.12307355e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.00000000e-01,
          1.15095056e+04],
        [ 1.24650194e+04, -7.97273525e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.75862276e-01,
          1.21602580e+04],
        [ 1.27294410e+04, -2.10000000e-01, -1.00000000e+00,
         -2.00000000e-01],
        [-2.81000000e-01, -2.11110000e-01,  1.24667302e+04,
         -1.00000000e+00]],

       [[ 1.00169719e+04, -2.46483670e+00, -3.00000000e+00,
         -2.48546079e+00],
        [ 1.07838732e+04, -2.53405180e+00, -2.46820563e+00,
         -2.90000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -6.07075799e-01,
          1.09491496e+04],
        [ 1.21443362e+04, -1.44214647e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.41846410e-01,
          1.07827362e+04],
        [ 1.21975070e+04, -1.05849541e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.71876114e-01,
          1.19251936e+04],
        [ 1.27280931e+04, -4.45235000e-01, -1.00000000e+00,
         -3.97200000e-01],
        [-3.72949736e-01, -4.41202200e-01,  1.21875225e+04,
         -1.00000000e+00]],

       [[-2.70774246e+00,  1.11204703e+04, -3.00000000e+00,
         -2.70077464e+00],
        [-2.60342267e+00,  1.15974131e+04, -2.60874288e+00,
         -3.00000000e+00],
        [-1.00000000e+00, -3.19911761e-01, -5.17580092e-03,
          1.02559052e+04],
        [ 1.16132948e+04, -1.84772771e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.88857738e-01, -2.75695864e-01,
          1.08581203e+04],
        [ 1.18337321e+04, -1.35260626e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.11000000e-01, -2.04155966e-01,
          1.19581574e+04],
        [ 1.27264828e+04, -6.49272976e-01, -1.00000000e+00,
         -6.82524512e-01],
        [-7.14766256e-01, -7.75729189e-01,  1.24875430e+04,
         -1.00000000e+00]],

       [[-2.65414882e+00, -2.64256734e+00, -3.00000000e+00,
          1.20592482e+04],
        [-2.52399105e+00, -2.55542226e+00, -2.50065094e+00,
          1.22448159e+04],
        [-3.00000000e+00, -2.30799186e+00, -2.25658255e+00,
          1.24013002e+04],
        [-2.02177372e+00, -2.03043489e+00, -2.01285684e+00,
          1.25572637e+04],
        [-2.00000000e+00, -1.80980087e+00, -1.84627467e+00,
          1.26294819e+04],
        [-1.48223860e+00, -1.53923914e+00, -1.53315082e+00,
          1.26773147e+04],
        [-2.00000000e+00, -1.21460623e+00, -1.27161941e+00,
          1.27089314e+04],
        [ 1.27227323e+04, -9.37341245e-01, -9.79858114e-01,
         -9.71433384e-01],
        [ 1.15567506e+04, -1.07913013e+00, -1.00329928e+00,
         -1.00000000e+00]],

       [[ 1.18037473e+04, -2.91519111e+00, -3.00000000e+00,
         -2.75355638e+00],
        [-2.70074531e+00, -3.10000000e+00, -2.73490191e+00,
          1.13261007e+04],
        [-2.44475859e+00, -3.00000000e+00, -2.46450980e+00,
          1.19616179e+04],
        [ 1.22706571e+04, -3.00000000e+00, -2.28589776e+00,
         -2.21441052e+00],
        [-1.94786644e+00, -1.98414187e+00, -1.93085185e+00,
          1.19406976e+04],
        [-1.64074328e+00, -2.10000000e+00, -1.65455737e+00,
          1.23484047e+04],
        [-1.38274167e+00, -2.07225501e+00, -1.38235753e+00,
          1.25587709e+04],
        [ 1.26884427e+04, -2.00000000e+00, -1.18141799e+00,
         -1.17672573e+00],
        [-1.22087955e+00, -2.00000000e+00,  1.22062315e+04,
         -2.00000000e+00]],

       [[ 1.11801105e+04, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 1.03419120e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.09709166e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00, -1.00000000e+00,
          1.01277800e+04],
        [ 1.14047377e+04, -3.00000000e+00, -3.00000000e+00,
         -3.00000000e+00],
        [ 1.11206887e+04, -1.00000000e+00, -2.77097787e-01,
         -1.00000000e+00],
        [ 1.09523649e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.21724679e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.13879658e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-2.00000000e+00,  7.28469677e+03, -2.00000000e+00,
         -1.17217764e+00],
        [-2.00000000e+00,  7.88528164e+03, -1.24315850e+00,
         -1.26295382e+00],
        [-2.00000000e+00,  7.75948393e+03, -1.13513681e+00,
         -1.18106702e+00],
        [-1.00000000e+00, -9.89136479e-01, -1.00438600e+00,
          7.53356373e+03],
        [-1.00000000e+00,  8.32744250e+03, -6.66071357e-01,
         -6.58085900e-01],
        [-1.00000000e+00,  7.98951337e+03, -5.36782570e-01,
         -5.40849000e-01],
        [-1.00000000e+00,  8.30966755e+03, -3.36910000e-01,
         -3.71900000e-01],
        [-1.00000000e+00,  7.91526341e+03, -2.28000000e-01,
         -2.81000000e-01],
        [-1.00000000e+00,  7.89370511e+03, -1.10000000e-01,
         -1.00000000e+00]],

       [[-1.14985225e+00,  8.11936180e+03, -2.00000000e+00,
         -1.18156111e+00],
        [-1.16637111e+00,  8.52884414e+03, -1.10995260e+00,
         -1.10629126e+00],
        [-1.03545756e+00,  8.65706229e+03, -1.11286810e+00,
         -1.02101596e+00],
        [-8.52344555e-01, -8.06998461e-01, -8.82755554e-01,
          8.26617469e+03],
        [-5.71758800e-01, -4.89566001e-01, -5.69003614e-01,
          8.81320894e+03],
        [-3.93710000e-01,  8.93552130e+03, -4.37505000e-01,
         -3.80000000e-01],
        [-2.91000000e-01,  8.84763378e+03, -2.09000000e-01,
         -2.00000000e-01],
        [-1.90000000e-01,  8.64875919e+03, -1.10000000e-01,
         -1.00000000e-01],
        [-1.00000000e-01,  8.84346707e+03,  0.00000000e+00,
          0.00000000e+00]],

       [[-1.25931388e+00, -1.26944202e+00, -2.00000000e+00,
          8.65814749e+03],
        [-1.10181782e+00, -1.06138254e+00, -1.10030326e+00,
          8.96176047e+03],
        [-9.88816334e-01, -1.00000000e+00, -1.03223505e+00,
          8.97353540e+03],
        [-7.69956038e-01, -7.75838916e-01, -7.20653864e-01,
          8.97597988e+03],
        [-3.91000000e-01, -1.10000000e+00, -4.59350700e-01,
          8.97708386e+03],
        [-2.00000000e-01, -1.90000000e-01, -2.10000000e-01,
          8.97809534e+03],
        [-1.00000000e-01, -1.00000000e+00, -1.10000000e-01,
          8.97909534e+03],
        [-1.00000000e-01,  8.98009534e+03,  0.00000000e+00,
          0.00000000e+00],
        [ 7.98209534e+03,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[-1.51793419e+00, -1.49099432e+00, -2.00000000e+00,
          8.65772513e+03],
        [ 8.92419266e+03, -1.37480111e+00, -1.32718487e+00,
         -2.00000000e+00],
        [ 8.23874069e+03,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 8.94223099e+03, -1.00707940e+00, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00, -1.10000000e-01,
          8.37130169e+03],
        [ 8.97691912e+03, -1.00000000e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00,  8.41448413e+03,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          8.98109534e+03],
        [ 8.98209534e+03,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[ 8.22785871e+03, -1.87292052e+00, -2.00000000e+00,
         -1.77102139e+00],
        [ 8.84956321e+03, -1.62183641e+00, -1.71710740e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -6.49426962e-01,
          7.90443838e+03],
        [ 8.84754514e+03, -1.24538306e+00, -2.01930779e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  8.34692412e+03,
          0.00000000e+00],
        [ 8.97416906e+03, -2.24904334e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  8.03600504e+03,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          8.14254834e+03],
        [ 8.98081028e+03,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[ 7.48656384e+03, -1.90388392e+00, -2.00000000e+00,
         -1.93546677e+00],
        [ 8.74936978e+03, -1.90546853e+00, -1.88851119e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -4.75319472e-01,
          7.91051623e+03],
        [ 8.59563885e+03, -1.41330674e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -6.17958724e-02,
          8.21007300e+03],
        [ 8.96594367e+03, -4.73243300e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.23733553e-01,
          7.83735716e+03],
        [-2.71000000e-01,  8.43542408e+03, -1.00000000e+00,
         -2.00000000e-01],
        [ 8.97729103e+03, -1.00000000e-01, -1.00000000e-01,
         -1.00000000e+00]],

       [[-2.10947548e+00, -2.19840924e+00, -3.00000000e+00,
          8.44298140e+03],
        [ 8.63750960e+03, -2.04434382e+00, -1.99857975e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -2.86876252e-01, -2.94172153e-01,
          6.70952640e+03],
        [ 7.85349860e+03, -1.58180546e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.99896072e-01, -2.20583040e-01,
          8.07697082e+03],
        [ 8.94545831e+03, -7.07378155e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.38910000e-01,  7.98616133e+03,
          0.00000000e+00],
        [-2.90000000e-01, -2.10000000e-01, -1.00000000e+00,
          8.86660606e+03],
        [ 8.96623271e+03, -2.10900000e-01, -2.10000000e-01,
         -1.00000000e+00]],

       [[ 8.17382383e+03, -2.49581698e+00, -3.00000000e+00,
         -2.41433929e+00],
        [ 7.79275796e+03, -2.27401849e+00, -2.29960680e+00,
         -2.26024405e+00],
        [-2.00000000e+00, -2.07988164e+00, -2.07850641e+00,
          8.47656926e+03],
        [-1.73333999e+00, -1.78269787e+00, -1.78780873e+00,
          8.64737040e+03],
        [-2.00000000e+00, -1.49947361e+00, -1.43800526e+00,
          8.78023830e+03],
        [ 8.90160447e+03, -1.05148152e+00, -1.12190391e+00,
         -1.01406406e+00],
        [-1.00000000e+00, -8.99613087e-01, -9.20567661e-01,
          7.60546693e+03],
        [ 8.50826595e+03, -5.32531500e-01, -5.54755730e-01,
         -5.23130004e-01],
        [ 8.90476486e+03, -4.32617000e-01, -4.14390000e-01,
         -1.00000000e+00]],

       [[ 7.80189963e+03, -3.00000000e+00, -3.00000000e+00,
         -2.60232783e+00],
        [-2.39343585e+00, -3.00000000e+00, -2.48761467e+00,
          7.77252425e+03],
        [ 8.27858397e+03, -3.00000000e+00, -2.16506055e+00,
         -2.22587568e+00],
        [-1.96712781e+00, -2.00000000e+00, -1.95100393e+00,
          7.77503252e+03],
        [-1.66395221e+00, -1.64315353e+00, -1.67420660e+00,
          8.59661914e+03],
        [ 8.77426062e+03, -2.10220090e+00, -1.27058415e+00,
         -1.26549410e+00],
        [-1.04346883e+00, -2.00000000e+00, -1.03795186e+00,
          8.21640071e+03],
        [-7.41675514e-01, -1.00000000e+00, -8.48805858e-01,
          8.58706722e+03],
        [ 8.76000868e+03, -1.00000000e+00, -6.47820737e-01,
         -1.00000000e+00]],

       [[ 7.12488033e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 6.89019589e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 7.77407825e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [-5.07577796e-01, -1.00000000e+00, -1.00000000e+00,
          7.42365545e+03],
        [ 8.17677462e+03, -2.00000000e+00, -2.00000000e+00,
         -2.02934883e+00],
        [ 8.07661260e+03, -1.00000000e+00, -5.46704990e-01,
         -1.00000000e+00],
        [ 7.00824718e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 7.66130849e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 7.77674104e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-2.00000000e+00,  9.50086401e+03, -2.00000000e+00,
         -1.19058693e+00],
        [-2.00000000e+00, -1.11492922e+00,  9.07680224e+03,
         -1.15686556e+00],
        [-2.00000000e+00, -1.07732954e+00, -1.12061762e+00,
          9.27206918e+03],
        [-1.00000000e+00,  9.63417229e+03, -9.35496789e-01,
         -8.82648605e-01],
        [-1.00000000e+00,  9.06982066e+03, -8.11329062e-01,
         -8.53721155e-01],
        [-1.00000000e+00,  9.63995956e+03, -6.46641780e-01,
         -6.31921000e-01],
        [-1.00000000e+00,  9.01995216e+03, -4.28954900e-01,
         -3.89100000e-01],
        [-1.00000000e+00,  9.55146075e+03, -3.10800000e-01,
         -3.72900000e-01],
        [-1.00000000e+00, -2.90000000e-01,  8.81501616e+03,
         -1.00000000e+00]],

       [[-1.08561264e+00, -1.10535501e+00, -2.00000000e+00,
          9.62906450e+03],
        [-1.08095365e+00,  9.74744481e+03, -1.03922036e+00,
         -1.07609889e+00],
        [-9.06433138e-01, -9.55921096e-01, -9.25682631e-01,
          9.15684965e+03],
        [-7.78712065e-01,  9.87488388e+03, -8.29943708e-01,
         -7.85462793e-01],
        [-6.89536535e-01, -5.89109000e-01, -6.37808921e-01,
          9.63505002e+03],
        [-4.92619000e-01,  9.94537457e+03, -4.25390000e-01,
         -3.98100000e-01],
        [-2.82000000e-01,  9.62851011e+03, -3.01000000e-01,
         -2.99000000e-01],
        [-2.92000000e-01,  9.87425432e+03, -2.09000000e-01,
         -1.90000000e-01],
        [-2.01000000e-01,  9.54518686e+03, -1.10000000e-01,
         -1.00000000e+00]],

       [[-1.08841801e+00, -1.15599453e+00, -2.00000000e+00,
          9.25289721e+03],
        [-1.07949750e+00, -1.07191321e+00, -1.04387548e+00,
          9.95404759e+03],
        [-9.07781906e-01, -1.10000000e+00, -9.45141841e-01,
          1.00423466e+04],
        [-6.95757930e-01, -6.97161202e-01, -7.30687921e-01,
          1.01006338e+04],
        [-5.02305000e-01, -1.00000000e+00, -5.11383030e-01,
          1.01209187e+04],
        [-3.00000000e-01, -2.71000000e-01, -2.19000000e-01,
          1.01280533e+04],
        [-2.00000000e-01, -1.00000000e+00, -2.00000000e-01,
          1.01292201e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.10000000e-01,
          1.01302551e+04],
        [-1.00000000e-01,  1.01312592e+04,  0.00000000e+00,
          0.00000000e+00]],

       [[-1.22392061e+00, -1.28332424e+00, -2.00000000e+00,
          9.65968856e+03],
        [ 9.83816784e+03, -1.15535378e+00, -1.18604860e+00,
         -2.00000000e+00],
        [ 9.37771658e+03, -1.00000000e+00, -1.00000000e-01,
         -1.78421284e-01],
        [ 9.92925065e+03, -8.21021177e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00,  9.38398617e+03,
          0.00000000e+00],
        [ 1.01219194e+04, -1.00000000e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00,  9.31916100e+03,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          9.76658548e+03],
        [-1.00000000e-01,  9.13125948e+03,  0.00000000e+00,
          0.00000000e+00]],

       [[ 9.35035243e+03, -1.52829398e+00, -2.00000000e+00,
         -1.45351415e+00],
        [ 9.71038558e+03, -1.40456511e+00, -1.37300358e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.16856132e+03,
          0.00000000e+00],
        [ 9.70748022e+03, -1.04912713e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  8.98851996e+03,
          0.00000000e+00],
        [ 1.01057758e+04, -1.00000000e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.55545701e+03,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          9.93176360e+03],
        [ 1.01288174e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[ 8.66510542e+03, -1.86052400e+00, -2.00000000e+00,
         -1.81067916e+00],
        [ 9.63505105e+03, -1.69882320e+00, -1.78658168e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.14141029e+03,
         -4.08727713e-01],
        [ 9.04517535e+03, -1.43507630e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.13706961e-01,
          9.09790030e+03],
        [ 1.00744442e+04, -4.47104025e-01, -1.00000000e+00,
         -1.10000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  9.49057608e+03,
         -2.00000000e-01],
        [ 9.70682958e+03, -2.00000000e-01, -1.11000000e+00,
         -1.90000000e-01],
        [ 1.01079410e+04, -1.00000000e-01, -1.10000000e-01,
         -1.00000000e+00]],

       [[-1.94449507e+00, -1.96746941e+00, -2.00000000e+00,
          9.22200708e+03],
        [ 9.53553357e+03, -1.95389269e+00, -1.90486083e+00,
         -2.00000000e+00],
        [-1.00000000e+00,  9.19668100e+03, -2.00000000e-01,
         -2.01182249e-01],
        [-1.55203343e+00,  9.30783420e+03, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.99136329e-01, -2.19957622e-01,
          9.35769082e+03],
        [ 1.00344528e+04, -6.93330046e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.84564448e-01,  9.19527276e+03,
          0.00000000e+00],
        [ 9.22181544e+03, -4.18910000e-01, -1.00000000e+00,
         -3.97200000e-01],
        [ 1.00531992e+04, -3.19100000e-01, -3.27000000e-01,
         -1.00000000e+00]],

       [[ 8.21315729e+03, -2.16701012e+00, -3.00000000e+00,
         -2.10400906e+00],
        [-1.98613460e+00, -2.03442184e+00, -1.99996418e+00,
          9.68102266e+03],
        [-2.00000000e+00, -1.77198902e+00, -1.86079744e+00,
          9.79829141e+03],
        [-1.54679415e+00, -1.53332464e+00, -1.49195449e+00,
          9.87934541e+03],
        [-2.00000000e+00, -1.28881889e+00, -1.27875788e+00,
          9.93833034e+03],
        [ 9.98988153e+03, -9.46021795e-01, -1.02153688e+00,
         -9.43882776e-01],
        [-1.00000000e+00, -8.96527775e-01, -8.68430795e-01,
          9.57138639e+03],
        [-6.37787100e-01, -5.99943300e-01, -6.40452400e-01,
          9.81559494e+03],
        [ 9.96807305e+03, -5.10072000e-01, -5.38970000e-01,
         -1.00000000e+00]],

       [[-2.31880431e+00, -3.00000000e+00, -3.00000000e+00,
          9.43791129e+03],
        [ 9.60479413e+03, -2.97381764e+00, -2.06666283e+00,
         -2.05163894e+00],
        [ 9.58052193e+03, -2.00000000e+00, -1.87076313e+00,
         -1.86229512e+00],
        [ 9.59584848e+03, -2.00000000e+00, -1.62729108e+00,
         -1.60256588e+00],
        [ 9.53295654e+03, -1.31794924e+00, -1.36543160e+00,
         -1.32027840e+00],
        [ 9.71998066e+03, -2.00000000e+00, -1.17336979e+00,
         -1.07116159e+00],
        [ 9.30507535e+03, -1.00000000e+00, -1.05203574e+00,
         -1.02918429e+00],
        [ 9.62817867e+03, -1.00000000e+00, -8.66874749e-01,
         -7.89061531e-01],
        [ 9.62030899e+03, -1.00000000e+00, -6.13479300e-01,
         -1.00000000e+00]],

       [[ 7.94785435e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.12078758e+03, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 8.75606808e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.11847483e+03, -1.00000000e+00, -1.00000000e+00,
         -4.04062009e-01],
        [ 9.03971074e+03, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 9.24569439e+03, -1.00000000e+00, -2.01168064e-01,
         -1.00000000e+00],
        [ 8.67326814e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.25541914e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 8.96097592e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-3.00000000e+00,  9.61079093e+03, -3.00000000e+00,
         -2.20220835e+00],
        [-2.00000000e+00, -2.00471567e+00, -2.03899422e+00,
          1.01023825e+04],
        [-2.00000000e+00, -1.72141011e+00, -1.66521434e+00,
          1.05305801e+04],
        [-2.00000000e+00,  1.07989242e+04, -1.32814832e+00,
         -1.34684466e+00],
        [-2.00000000e+00,  1.04571609e+04, -1.08282320e+00,
         -1.07853474e+00],
        [-1.00000000e+00, -8.50626909e-01, -8.72602792e-01,
          1.03627789e+04],
        [-1.00000000e+00,  1.09496347e+04, -6.48534328e-01,
         -5.69470000e-01],
        [-1.00000000e+00,  1.04832382e+04, -4.26930000e-01,
         -4.65510000e-01],
        [-1.00000000e+00,  1.04029702e+04, -3.28090000e-01,
         -1.00000000e+00]],

       [[-2.09960143e+00, -2.17079325e+00, -3.00000000e+00,
          1.03459809e+04],
        [-2.00451915e+00, -1.96909260e+00, -1.99598713e+00,
          1.06789152e+04],
        [-1.64845801e+00, -1.59339189e+00, -1.58126008e+00,
          1.08536797e+04],
        [-1.26627635e+00, -1.24072110e+00, -1.22220963e+00,
          1.10257822e+04],
        [-9.62417387e-01, -9.87083120e-01, -1.05812483e+00,
          1.10994431e+04],
        [-8.51032088e-01, -8.02161528e-01, -8.42236990e-01,
          1.11567839e+04],
        [-5.72415100e-01, -4.96750000e-01, -5.56373400e-01,
          1.12108160e+04],
        [-3.81100000e-01,  1.12821221e+04, -4.17071500e-01,
         -3.91000000e-01],
        [-2.82000000e-01,  1.10434307e+04, -3.27389000e-01,
         -1.00000000e+00]],

       [[-2.20275840e+00, -2.23229442e+00, -3.00000000e+00,
          1.03980196e+04],
        [-2.00124789e+00, -1.98568348e+00, -2.00619137e+00,
          1.10123103e+04],
        [-1.61320257e+00, -2.00000000e+00, -1.61715694e+00,
          1.11144819e+04],
        [-1.22378039e+00, -1.17915293e+00, -1.21769772e+00,
          1.11613463e+04],
        [-9.84379075e-01, -1.00000000e+00, -9.34160220e-01,
          1.11854665e+04],
        [-8.60723912e-01, -8.51394645e-01, -8.13050198e-01,
          1.12251459e+04],
        [-4.92430000e-01, -1.00000000e+00, -5.64660368e-01,
          1.12921488e+04],
        [-3.09000000e-01, -2.71000000e-01, -2.20910000e-01,
          1.13846233e+04],
        [-2.01000000e-01,  1.14328520e+04, -1.10000000e-01,
         -1.00000000e+00]],

       [[-2.31070516e+00, -2.30461481e+00, -3.00000000e+00,
          1.06602555e+04],
        [ 1.08658502e+04, -2.22154924e+00, -2.25490748e+00,
         -3.00000000e+00],
        [ 1.04023127e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.10836897e+04, -1.15016945e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.40000000e-01, -1.00000000e+00,  1.03407022e+04,
          0.00000000e+00],
        [ 1.06187316e+04, -1.05609728e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00, -1.95576733e-01,
          1.03940090e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          1.10387477e+04],
        [-1.00000000e-01,  1.14624650e+04,  0.00000000e+00,
          0.00000000e+00]],

       [[ 1.00975903e+04, -2.47225401e+00, -3.00000000e+00,
         -2.52945298e+00],
        [ 1.03577539e+04, -2.50596191e+00, -2.54286434e+00,
         -3.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -3.83996958e-01,
          1.02518666e+04],
        [ 1.09385938e+04, -1.42453519e+00, -2.01961965e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.02647892e+04,
          0.00000000e+00],
        [-1.22935170e+00,  1.04002026e+04, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.14261468e-01,
          1.03560259e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          1.10611434e+04],
        [ 1.04718134e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[-2.60655448e+00, -2.59013689e+00, -3.00000000e+00,
          9.92048939e+03],
        [-2.50779508e+00,  1.05211877e+04, -2.49651319e+00,
         -3.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -3.37157005e-01,
          9.31582500e+03],
        [ 1.06212405e+04, -1.73864545e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.47117851e-01,
          1.00313017e+04],
        [-1.15901894e+00,  1.08498338e+04, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.98608688e-01,
          1.01201964e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          1.10822665e+04],
        [ 1.14619432e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[-2.51104081e+00, -2.48161189e+00, -3.00000000e+00,
          1.03519810e+04],
        [-2.48699944e+00,  1.07859251e+04, -2.52490183e+00,
         -3.00000000e+00],
        [-1.00000000e+00, -4.61909766e-01, -5.25316081e-01,
          1.04562146e+04],
        [-1.64527597e+00,  1.09719612e+04, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e-01,  1.04419291e+04,
          0.00000000e+00],
        [-1.09256291e+00,  1.11076110e+04, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -5.03909137e-02,  1.06743853e+04,
          0.00000000e+00],
        [-1.90000000e-01, -2.19900000e-01, -1.00000000e+00,
          1.09199757e+04],
        [ 1.14356434e+04, -1.00000000e-01, -1.00000000e-01,
         -1.00000000e+00]],

       [[-2.38762155e+00, -2.42893317e+00, -3.00000000e+00,
          1.04377680e+04],
        [-2.27012618e+00, -2.26445664e+00, -2.23227511e+00,
          1.10345034e+04],
        [-2.00000000e+00, -1.97111831e+00, -2.01365147e+00,
          1.11175777e+04],
        [-1.58687454e+00, -1.62462116e+00, -1.64261533e+00,
          1.11815969e+04],
        [-2.00000000e+00, -1.20202337e+00, -1.27122228e+00,
          1.12345946e+04],
        [-1.03677018e+00, -1.03038923e+00, -9.80022914e-01,
          1.12813343e+04],
        [-1.00000000e+00, -7.13532255e-01, -6.66124391e-01,
          1.13243667e+04],
        [-4.36610000e-01, -4.00000000e-01, -4.45029000e-01,
          1.13646034e+04],
        [ 1.14017998e+04, -3.09100000e-01, -3.27100000e-01,
         -1.00000000e+00]],

       [[-2.45728107e+00, -3.00000000e+00, -3.00000000e+00,
          1.06802101e+04],
        [ 1.08917168e+04, -3.00000000e+00, -2.34445447e+00,
         -2.36115555e+00],
        [-2.04806351e+00, -2.97091446e+00, -2.03375461e+00,
          1.06117240e+04],
        [-1.66493332e+00, -2.00000000e+00, -1.74736956e+00,
          1.08744872e+04],
        [ 1.10768242e+04, -1.40488874e+00, -1.38957221e+00,
         -1.40978128e+00],
        [-1.11315128e+00, -2.00000000e+00, -1.16059606e+00,
          1.06547995e+04],
        [-7.76290838e-01, -1.00000000e+00, -8.76598558e-01,
          1.08986180e+04],
        [-5.41243200e-01, -1.00000000e+00, -5.64605035e-01,
          1.10747526e+04],
        [ 1.12210244e+04, -1.00000000e+00, -5.20550520e-01,
         -1.00000000e+00]],

       [[ 1.00306352e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.00300363e+04, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 9.96554896e+03, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-4.00066216e-01, -1.00000000e+00, -1.00000000e+00,
          9.87974739e+03],
        [ 1.06666999e+04, -2.00000000e+00, -2.00000000e+00,
         -2.00000000e+00],
        [ 1.02174517e+04, -1.00000000e+00, -1.33799000e-01,
         -1.00000000e+00],
        [ 1.01800900e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.06529245e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.04992785e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]]), array([[[-2.00000000e+00, -1.79984993e+00, -2.00000000e+00,
          1.09126328e+04],
        [-2.00000000e+00,  1.19706454e+04, -1.63387167e+00,
         -1.66120572e+00],
        [-2.00000000e+00,  1.05150043e+04, -1.44494203e+00,
         -1.49632397e+00],
        [-2.00000000e+00, -1.19550672e+00, -1.24047520e+00,
          1.16020711e+04],
        [-1.00000000e+00,  1.25908851e+04, -9.71822086e-01,
         -9.77575519e-01],
        [-1.00000000e+00,  1.15859946e+04, -6.64456227e-01,
         -6.53143900e-01],
        [-1.00000000e+00, -3.90900000e-01, -4.36901000e-01,
          1.12327370e+04],
        [-1.00000000e+00,  1.27238413e+04, -2.20000000e-01,
         -2.90000000e-01],
        [-1.00000000e+00,  1.20707660e+04, -2.11800000e-01,
         -1.00000000e+00]],

       [[-1.71885947e+00, -1.65908341e+00, -2.00000000e+00,
          1.15622413e+04],
        [-1.51898825e+00,  1.26201809e+04, -1.60616473e+00,
         -1.54969306e+00],
        [-1.33815018e+00, -1.36660176e+00, -1.33344055e+00,
          1.15219792e+04],
        [-1.09160081e+00, -1.05225531e+00, -1.06313530e+00,
          1.23576250e+04],
        [-7.62613891e-01, -7.75731798e-01, -7.24217000e-01,
          1.30522216e+04],
        [-4.65800000e-01, -4.82090000e-01, -5.33105690e-01,
          1.33156700e+04],
        [-2.81000000e-01, -2.97589000e-01, -3.19010000e-01,
          1.35070325e+04],
        [-2.82000000e-01,  1.36122364e+04, -2.01000000e-01,
         -2.00000000e-01],
        [-1.90000000e-01,  1.30894857e+04, -1.00000000e-01,
         -1.00000000e+00]],

       [[-1.68801014e+00, -1.69748899e+00, -2.00000000e+00,
          1.10114363e+04],
        [-1.54548916e+00, -1.62962885e+00, -1.53872968e+00,
          1.31254112e+04],
        [-1.31323328e+00, -2.00000000e+00, -1.31233036e+00,
          1.33010400e+04],
        [-1.02175711e+00, -1.03675421e+00, -9.69603755e-01,
          1.34636417e+04],
        [-7.07103428e-01, -1.00000000e+00, -6.37497859e-01,
          1.35519645e+04],
        [-3.99990000e-01, -3.71900000e-01, -4.27101000e-01,
          1.36131932e+04],
        [-2.00000000e-01, -1.10000000e+00, -2.09000000e-01,
          1.36390728e+04],
        [-1.90000000e-01, -1.90000000e-01, -2.00000000e-01,
          1.36523913e+04],
        [-1.90000000e-01,  1.36564175e+04, -1.10000000e-01,
         -1.00000000e+00]],

       [[-1.92643145e+00, -1.95314112e+00, -2.00000000e+00,
          1.22181662e+04],
        [ 1.27496648e+04, -1.90101685e+00, -1.90231554e+00,
         -2.00000000e+00],
        [ 1.10375577e+04, -1.00000000e+00, -4.31033062e-01,
         -3.83498473e-01],
        [ 1.29586593e+04, -1.12250097e+00, -2.01110000e+00,
         -2.00000000e+00],
        [-1.68131444e-01, -1.00000000e+00, -3.00110000e-01,
          1.18215716e+04],
        [ 1.33811001e+04, -3.62900000e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00,  1.20830968e+04,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          1.27146202e+04],
        [-1.00000000e-01,  1.36716039e+04,  0.00000000e+00,
          0.00000000e+00]],

       [[ 1.15310797e+04, -1.99074586e+00, -2.00000000e+00,
         -2.04343027e+00],
        [ 1.14237516e+04, -1.93965752e+00, -2.00992143e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.90599429e-01,
          1.11291032e+04],
        [ 1.23604667e+04, -1.09955252e+00, -2.00000000e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.00000000e-01,
          1.15321041e+04],
        [ 1.29300614e+04, -2.90000000e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.21000000e-01,
          1.20007768e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          1.29957572e+04],
        [-1.00000000e-01,  1.37352384e+04,  0.00000000e+00,
          0.00000000e+00]],

       [[ 1.06126997e+04, -1.97148019e+00, -2.00000000e+00,
         -1.94822062e+00],
        [-1.87629242e+00,  1.20592759e+04, -1.83694490e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.11147010e+04,
         -2.02115390e-01],
        [ 1.14554567e+04, -1.11877315e+00, -2.00000000e+00,
         -2.01210000e+00],
        [-1.00000000e+00, -1.00000000e+00, -2.90000000e-01,
          1.12532115e+04],
        [ 1.23661730e+04, -3.01900000e-01, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.13270886e+04,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          1.22587392e+04],
        [-1.00000000e-01,  1.28479087e+04,  0.00000000e+00,
          0.00000000e+00]],

       [[-1.88752049e+00, -1.83308003e+00, -2.00000000e+00,
          1.18606118e+04],
        [-1.69240491e+00,  1.26816630e+04, -1.73336972e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.87381022e-01,  1.12836710e+04,
          0.00000000e+00],
        [-1.09058897e+00,  1.22971358e+04, -1.00000000e+00,
         -1.10000000e+00],
        [-1.00000000e+00, -1.00000000e-01, -1.10000000e-01,
          1.17050292e+04],
        [-2.91000000e-01,  1.30029024e+04, -1.00000000e+00,
         -1.10000000e+00],
        [-1.00000000e+00, -1.00000000e-01, -1.00000000e-01,
          1.25526867e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.00000000e+00,
          1.36557793e+04],
        [ 1.36851515e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[-1.69938839e+00, -1.74180770e+00, -2.00000000e+00,
          1.17688921e+04],
        [-1.45975811e+00, -1.54060892e+00, -1.47161218e+00,
          1.31228961e+04],
        [-2.00000000e+00, -1.19503296e+00, -1.21828494e+00,
          1.33352834e+04],
        [-8.73460473e-01, -8.80826684e-01, -9.56627994e-01,
          1.35017606e+04],
        [-1.00000000e+00, -5.96620410e-01, -5.66489825e-01,
          1.35771333e+04],
        [-3.62900000e-01, -3.01900000e-01, -3.42032380e-01,
          1.36225867e+04],
        [-1.00000000e+00, -2.00000000e-01, -2.01000000e-01,
          1.36436120e+04],
        [ 1.36530284e+04, -1.00000000e-01, -1.00000000e-01,
         -1.00000000e-01],
        [ 1.24476758e+04, -1.00000000e-01, -1.10000000e-01,
         -1.00000000e+00]],

       [[-1.65658048e+00, -2.10000000e+00, -2.00000000e+00,
          1.19552601e+04],
        [ 1.25878966e+04, -2.00000000e+00, -1.58011798e+00,
         -1.52896823e+00],
        [ 1.23468324e+04, -2.10000000e+00, -1.29112205e+00,
         -1.18928549e+00],
        [ 1.28106208e+04, -1.00000000e+00, -9.48047867e-01,
         -9.77605202e-01],
        [ 1.29049215e+04, -7.74931700e-01, -7.82762387e-01,
         -7.10487414e-01],
        [-3.82900000e-01, -1.00000000e+00, -4.45706737e-01,
          1.21714440e+04],
        [-2.00000000e-01, -1.00000000e+00, -2.00000000e-01,
          1.29757030e+04],
        [ 1.35114089e+04, -1.00000000e-01, -1.00000000e-01,
         -1.00000000e-01],
        [-1.00000000e-01, -1.00000000e+00,  1.25699943e+04,
          0.00000000e+00]],

       [[ 1.10779155e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.15001555e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.07039628e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.05555572e+04, -1.00000000e+00, -1.00000000e+00,
         -1.85068724e-01],
        [ 1.20505136e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.12946657e+04, -1.00000000e+00, -1.78706529e-01,
         -1.00000000e+00],
        [ 1.11921812e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 1.20551723e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.12262454e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]]]), array([[[-3.00000000e+00, -2.43737869e+00, -3.00000000e+00,
          9.49982418e+03],
        [-3.00000000e+00,  1.03150137e+04, -2.32977784e+00,
         -2.36716285e+00],
        [-3.00000000e+00,  9.70359631e+03, -2.20273891e+00,
         -2.19683716e+00],
        [-2.00000000e+00,  1.03816875e+04, -2.09105026e+00,
         -2.06021956e+00],
        [-3.00000000e+00, -2.04183915e+00,  9.31905426e+03,
         -2.06194263e+00],
        [-2.00000000e+00,  1.10040408e+04, -2.04122042e+00,
         -2.01394817e+00],
        [-3.00000000e+00, -2.08585548e+00,  1.06621595e+04,
         -2.14907567e+00],
        [-3.00000000e+00, -2.31374755e+00,  1.03543814e+04,
         -2.26535571e+00],
        [-3.00000000e+00, -2.37976378e+00,  9.89797840e+03,
         -3.00000000e+00]],

       [[-2.36529415e+00,  9.64851425e+03, -3.00000000e+00,
         -2.38622973e+00],
        [-2.19477544e+00,  1.08234456e+04, -2.17665198e+00,
         -2.17167279e+00],
        [-2.09260954e+00, -2.01463911e+00, -2.08791348e+00,
          1.04798635e+04],
        [-1.89288207e+00,  1.10015468e+04, -1.88670771e+00,
         -1.86637796e+00],
        [-1.96499592e+00, -1.93802045e+00, -1.93177361e+00,
          1.00001527e+04],
        [-1.89277308e+00,  1.13237779e+04, -1.87121294e+00,
         -1.91281794e+00],
        [-2.09546982e+00, -2.03915804e+00,  1.09950497e+04,
         -2.04746865e+00],
        [-2.20888011e+00, -2.22791004e+00,  1.01361304e+04,
         -2.24872814e+00],
        [ 8.61082660e+03, -2.25341405e+00, -2.26467082e+00,
         -3.00000000e+00]],

       [[-2.12290286e+00, -2.07395720e+00, -3.00000000e+00,
          1.07141037e+04],
        [-1.88314930e+00,  1.11946416e+04, -1.91503630e+00,
         -1.93602962e+00],
        [-1.88993528e+00, -2.00000000e+00, -1.85016959e+00,
          1.02698540e+04],
        [-1.57372787e+00,  1.13083537e+04, -1.59981099e+00,
         -1.60948086e+00],
        [-1.77733056e+00, -2.00000000e+00,  9.77558194e+03,
         -1.79002221e+00],
        [-1.63207170e+00,  1.14211623e+04, -1.60456679e+00,
         -1.66475606e+00],
        [-1.88119421e+00, -2.00000000e+00,  1.05740436e+04,
         -1.86950850e+00],
        [-2.10828125e+00,  9.80716526e+03, -2.03138767e+00,
         -2.07127984e+00],
        [-2.10358759e+00,  9.36540322e+03, -2.10708823e+00,
         -3.00000000e+00]],

       [[-1.82361863e+00, -1.73712785e+00, -2.00000000e+00,
          1.03024830e+04],
        [-1.64587440e+00,  1.13756274e+04, -1.61359980e+00,
         -2.00000000e+00],
        [-1.20000000e-01, -1.00000000e+00, -1.64266627e-01,
          1.03138411e+04],
        [-1.08940263e+00,  1.14641496e+04, -2.00000000e+00,
         -2.00000000e+00],
        [-2.73767705e-01, -1.00000000e+00,  1.03869648e+04,
          0.00000000e+00],
        [-1.20465184e+00,  1.14972415e+04, -2.00000000e+00,
         -2.00000000e+00],
        [-2.16063317e-01, -1.00000000e+00,  1.03948127e+04,
         -2.66182582e-01],
        [-1.80776222e+00,  1.06927325e+04, -2.00000000e+00,
         -1.71667799e+00],
        [-1.85710864e+00, -1.82596626e+00,  1.02213140e+04,
         -2.00000000e+00]],

       [[-1.44839569e+00, -1.43200334e+00, -2.00000000e+00,
          1.03986291e+04],
        [-1.32070645e+00,  1.14944257e+04, -1.30537005e+00,
         -2.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.05783184e+04,
          0.00000000e+00],
        [-8.21121811e-01,  1.15488649e+04, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00, -1.67190543e-01,
          1.01874464e+04],
        [-8.31780270e-01,  1.15586506e+04, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.06147963e+04,
          0.00000000e+00],
        [-1.46081853e+00,  1.10477194e+04, -2.00000000e+00,
         -1.34891666e+00],
        [-1.55721022e+00,  1.01232196e+04, -1.46221762e+00,
         -2.00000000e+00]],

       [[-1.17391124e+00, -1.15386469e+00, -2.00000000e+00,
          1.04951622e+04],
        [-1.08397289e+00,  1.15628096e+04, -1.01929854e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.04211949e+04,
          0.00000000e+00],
        [-4.59705939e-01,  1.16057329e+04, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.09098815e+04,
          0.00000000e+00],
        [-4.70697064e-01,  1.16112142e+04, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00, -1.00000000e+00,  1.07711129e+04,
          0.00000000e+00],
        [-1.08730023e+00,  1.13993981e+04, -2.00000000e+00,
         -1.07008655e+00],
        [-1.15112067e+00, -1.19688906e+00,  1.10674966e+04,
         -2.00000000e+00]],

       [[-9.87378199e-01, -9.16508322e-01, -1.00000000e+00,
          1.03707589e+04],
        [-7.52338092e-01,  1.16078504e+04, -8.25632091e-01,
         -1.00000000e+00],
        [-1.00000000e+00,  1.02510358e+04,  0.00000000e+00,
          0.00000000e+00],
        [-2.91000000e-01,  1.16357897e+04, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00,  1.05051167e+04,  0.00000000e+00,
          0.00000000e+00],
        [-2.12348978e-01,  1.16406174e+04, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e+00,  1.09420359e+04,  0.00000000e+00,
          0.00000000e+00],
        [-7.58839750e-01,  1.15133935e+04, -1.00000000e+00,
         -8.29833773e-01],
        [-9.07330895e-01, -9.17472514e-01,  1.06115722e+04,
         -1.00000000e+00]],

       [[-7.38475753e-01,  1.07708324e+04, -1.00000000e+00,
         -6.26654501e-01],
        [-4.96571280e-01,  1.16374660e+04, -5.21742200e-01,
         -5.46978000e-01],
        [-1.00000000e+00,  1.13079043e+04, -3.18900000e-01,
         -2.98100000e-01],
        [-1.90000000e-01, -1.90000000e-01, -2.09000000e-01,
          1.16514369e+04],
        [-1.00000000e+00,  1.16574930e+04, -1.00000000e-01,
         -1.00000000e-01],
        [-1.90000000e-01,  1.16544132e+04, -1.10000000e-01,
         -1.00000000e-01],
        [-1.00000000e+00,  1.16405142e+04, -2.09000000e-01,
         -2.10000000e-01],
        [-5.32605912e-01, -4.58510000e-01,  1.15931094e+04,
         -4.33580000e-01],
        [-7.40464931e-01, -6.52610200e-01,  1.04538825e+04,
         -1.00000000e+00]],

       [[-4.46336890e-01, -1.00000000e+00, -1.00000000e+00,
          1.14648635e+04],
        [-3.18100000e-01, -1.00000000e+00, -3.00900000e-01,
          1.16518720e+04],
        [-2.00000000e-01, -1.10000000e+00, -2.10000000e-01,
          1.16565256e+04],
        [-1.00000000e-01, -1.00000000e-01, -1.10000000e-01,
          1.16582360e+04],
        [-1.00000000e-01,  1.16593217e+04,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e-01,  1.16583161e+04,
          0.00000000e+00],
        [-2.00000000e-01, -1.00000000e+00,  1.16560916e+04,
         -1.10000000e-01],
        [-3.10000000e-01, -1.10000000e+00,  1.15506842e+04,
         -3.43480000e-01],
        [-5.24481000e-01, -1.00000000e+00,  1.08321296e+04,
         -1.00000000e+00]],

       [[ 1.05775719e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.05681775e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.06752165e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00, -1.00000000e+00,
          1.05027372e+04],
        [ 1.06593217e+04,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-1.00000000e-01, -1.00000000e+00,  1.06216226e+04,
          0.00000000e+00],
        [ 1.01885313e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 1.07860433e+04, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00],
        [ 9.70733258e+03, -1.00000000e+00, -1.00000000e+00,
         -1.00000000e+00]]])]

NameError: ignored

하드 코딩으로 Q-Tables를 저장해두고 필요하면 써보려 했으나 실패

In [74]:
class Simulator:
    def __init__(self):
        '''
        height : 그리드 높이
        width : 그리드 너비 
        inds : A ~ Q alphabet list
        '''
        # Load train data
        self.files = pd.read_csv(os.path.join(local_path, "./data/factory_order_test.csv"))
        self.height = 10 #세로 10
        self.width = 9 # 가로 9  #A~Q까지 알파벳(target) 선언
        self.failcount = 0
        self.successcount = 0
    def set_box(self):
        '''
        아이템들이 있을 위치를 미리 정해놓고 그 위치 좌표들에 아이템이 들어올 수 있으므로 그리드에 100으로 표시한다.
        데이터 파일에서 이번 에피소드 아이템 정보를 받아 가져와야 할 아이템이 있는 좌표만 -100으로 표시한다.
        self.local_target에 에이전트가 이번에 방문해야할 좌표들을 저장한다.
        따라서 가져와야하는 아이템 좌표와 end point 좌표(처음 시작했던 좌표로 돌아와야하므로)가 들어가게 된다.
        '''
        box_data = pd.read_csv(os.path.join(local_path, "./data/box.csv"))

        # 물건이 들어있을 수 있는 경우
        for box in box_data.itertuples(index = True, name ='Pandas'):  #판다스 데이터를 튜플로 iter해준 것 같다. 행/열 위치정보로 각 알파벳이 표시됨
            self.grid[getattr(box, "row")][getattr(box, "col")] = 100   #아이템이 없는 경우 100

        # 물건이 실제 들어있는 경우
        order_item = list(set(inds) & set(self.items))
        order_csv = box_data[box_data['item'].isin(order_item)]
        for order_box in order_csv.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = -100 # 아이템이 있는 경우 -100 <- 값이 지정되면 리턴으로 사용가능?
            # local target에 가야 할 위치 좌표 넣기
            self.local_target.append(
                [getattr(order_box, "row"),
                 getattr(order_box, "col")]
                )# 타겟 위치 지정 완료
            # print(order_box)
            
        # print(self.local_target)
        self.local_target.append([9,4])  # 마지막 타겟 = 출발위치

        # 알파벳을 Grid에 넣어서 -> grid에 2Dconv 적용 가능

    def set_obstacle(self):
        '''
        장애물이 있어야하는 위치는 미리 obstacles.csv에 정의되어 있다. 이 좌표들을 0으로 표시한다.
        '''
        obstacles_data = pd.read_csv(os.path.join(local_path, "./data/obstacles.csv"))
        for obstacle in obstacles_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(obstacle, "row")][getattr(obstacle, "col")] = 0 #장애물 행,열 지정

    def reset(self, epi):
        '''
        reset()은 첫 스텝에서 사용되며 그리드에서 에이전트 위치가 start point에 있게 한다.

        :param epi: episode, 에피소드 마다 가져와야 할 아이템 리스트를 불러올 때 사용
        :return: 초기셋팅 된 그리드
        :rtype: numpy.ndarray
        _____________________________________________________________________________________
        items : 이번 에피소드에서 가져와야하는 아이템들
        terminal_location : 현재 에이전트가 찾아가야하는 목적지
        local_target : 한 에피소드에서 찾아가야하는 아이템 좌표, 마지막 엔드 포인트 등의 위치좌표들
        actions: visualization을 위해 에이전트 action을 저장하는 리스트
        curloc : 현재 위치
        '''

        # initial episode parameter setting
        self.epi = epi
        self.items = list(self.files.iloc[self.epi])[0]
        self.cumulative_reward = 0
        self.terminal_location = None
        self.local_target = []
        self.actions = [] # 정책에 따라 시간 순서대로 action 리스트에 append해주면 될듯

        # initial grid setting
        self.grid = np.ones((self.height, self.width), dtype="float16") #초기는 전부0

        # set information about the gridworld
        self.set_box() #빈박스 + 타겟 박스
        self.set_obstacle() #장애물

        # start point를 grid에 표시
        self.curloc = [9, 4] #시작 위치 초기화 (인덱스 0부터 시작이므로 10번째줄, 5번째 열임)
        self.grid[int(self.curloc[0])][int(self.curloc[1])] = -5       
        
        self.done = False
        
        return self.grid

    def apply_action(self, action, cur_x, cur_y):
        '''
        에이전트가 행한 action대로 현 에이전트의 위치좌표를 바꾼다.
        action은 discrete하며 4가지 up,down,left,right으로 정의된다.
        
        :param x: 에이전트의 현재 x 좌표
        :param y: 에이전트의 현재 y 좌표
        :return: action에 따라 변한 에이전트의 x 좌표, y 좌표
        :rtype: int, int
        '''
        new_x = cur_x
        new_y = cur_y
        # up
        if action == 0:
            new_x = cur_x - 1
        # down
        elif action == 1:
            new_x = cur_x + 1
        # left
        elif action == 2:
            new_y = cur_y - 1
        # right
        else:
            new_y = cur_y + 1

        return int(new_x), int(new_y)


    def get_reward(self, new_x, new_y, out_of_boundary):
        '''
        get_reward함수는 리워드를 계산하는 함수이며, 상황에 따라 에이전트가 action을 옳게 했는지 판단하는 지표가 된다.

        :param new_x: action에 따른 에이전트 새로운 위치좌표 x
        :param new_y: action에 따른 에이전트 새로운 위치좌표 y
        :param out_of_boundary: 에이전트 위치가 그리드 밖이 되지 않도록 제한
        :return: action에 따른 리워드
        :rtype: float
        '''

        # 바깥으로 나가는 경우
        if any(out_of_boundary):
            reward = obs_reward
                       
        else:
            # 장애물에 부딪히는 경우 
            if self.grid[new_x][new_y] == 0:
                reward = obs_reward  

            # 현재 목표에 도달한 경우
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:
                reward = goal_reward

            # 그냥 움직이는 경우 
            else:
                reward = move_reward

        return reward

    def step(self, action):
        ''' 
        에이전트의 action에 따라 step을 진행한다.
        action에 따라 에이전트 위치를 변환하고, action에 대해 리워드를 받고, 어느 상황에 에피소드가 종료되어야 하는지 등을 판단한다.
        에이전트가 endpoint에 도착하면 gif로 에피소드에서 에이전트의 행동이 저장된다.

        :param action: 에이전트 행동
        :return:
            grid, 그리드
            reward, 리워드
            cumulative_reward, 누적 리워드
            done, 종료 여부
            goal_ob_reward, goal까지 아이템을 모두 가지고 돌아오는 finish율 계산을 위한 파라미터

        :rtype: numpy.ndarray, float, float, bool, bool/str

        (Hint : 시작 위치 (9,4)에서 up말고 다른 action은 전부 장애물이므로 action을 고정하는 것이 좋음)
        '''
        # print(self.local_target)
        self.terminal_location = self.local_target[0]
        cur_x,cur_y = self.curloc
        self.actions.append((cur_x, cur_y))

        goal_ob_reward = False
        
        new_x, new_y = self.apply_action(action, cur_x, cur_y)

        out_of_boundary = [new_x < 0, new_x >= self.height, new_y < 0, new_y >= self.width]

        # 바깥으로 나가는 경우 종료
        if any(out_of_boundary):
            self.done = True
            goal_ob_reward = True
            self.failcount += 1
        else:
            # 장애물에 부딪히는 경우 종료
            if self.grid[new_x][new_y] == 0:
                self.done = True
                goal_ob_reward = True
                self.failcount += 1

            # 현재 목표에 도달한 경우, 다음 목표설정
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:
                

                # end point 일 때
                if [new_x, new_y] == [9,4]:
                    self.done = True # 마지막 목표가 설정되면 done을 True로 설정
                    self.successcount +=1

                self.local_target.remove(self.local_target[0]) # 목표 달성시 목표 맨 앞 리스트를 지워준다.
                self.grid[cur_x][cur_y] = 1
                self.grid[new_x][new_y] = -5
                goal_ob_reward = True
                self.curloc = [new_x, new_y]
            else:
                # 그냥 움직이는 경우 
                self.grid[cur_x][cur_y] = 1
                self.grid[new_x][new_y] = -5
                self.curloc = [new_x,new_y]
                
        reward = self.get_reward(new_x, new_y, out_of_boundary)
        self.cumulative_reward += reward

        # if self.done == True:
        #     if [new_x, new_y] == [9, 4]: #초기위치로 도달했을 때,
        #         if self.terminal_location == [9, 4]: # 목표와 같을때만 gif로 만들어줌
        #             # 완료되면 GIFS 저장
        #             goal_ob_reward = 'finish'
        #             height = 10
        #             width = 9
        #             display = Display(visible=False, size=(width, height))
        #             display.start()
        #
        #             start_point = (9, 4)
        #             unit = 50
        #             screen_height = height * unit
        #             screen_width = width * unit
        #             log_path = "./logs"
        #             data_path = "./data"
        #             render_cls = Render(screen_width, screen_height, unit, start_point, data_path, log_path)
        #             for idx, new_pos in enumerate(self.actions):
        #                 render_cls.update_movement(new_pos, idx+1)
        #
        #             render_cls.save_gif(self.epi)
        #             render_cls.viewer.close()
        #             display.stop()
        
        return self.grid, reward, self.cumulative_reward, self.done, goal_ob_reward



            


환경은 원본코드 거의 그대로 사용, 

전역변수 선언해준 것

factory_order_test만 불러옴

사실 완성 Q-Table이 있으니 test나 train이나 그게 그거임

In [27]:
class QAgent_exploitation(): #테스트 클래스, 마찬가지로 입실론은 안써도 된다.
    def __init__(self):
        self.eps = 0.0  # exploitation만 하면됨
        self.eps_decay = 0.00

    
    def select_action(self, curloc, index):
        # index인수로 받아서 접근만 추가함
        x, y = curloc
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = Q_tables[index][x,y,:]
            action = np.argmax(action_val)
        return action

    def update_table(self, transition, index):
        s, a, r , s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime, index) 
        # Q_tables[index][x,y,a] = Q_tables[index][x,y,a] + 0.1*(r + np.max(Q_tables[index][next_x, next_y, :]) - Q_tables[index][x,y,a])
        # table은 더이상 업데이트 안해도됨, Q-Table만 따라 이동하면 된다.


    def anneal_eps(self):
        self.eps -= self.eps_decay
        self.eps = max(self.eps, 0.0)

In [78]:
def main():
    env = Simulator()
    agent = QAgent_exploitation()
    episode = len(env.files)

    for n_epi in range(episode):
        alst = []
        acount = 0
        done = False
        env.reset(n_epi)
        while not done: # 에피소드가 끝날때 까지 반복
            targetidx = box_coor_index.index(env.local_target[0])
            s = env.curloc
            a = agent.select_action(s, targetidx)
            alst.append(a) #액션들을 alst에 모아둠 -> 나중에 시각화하려구
            acount += 1 #액션 수
            _, r, cum_reward, done, goal_ob_reward = env.step(a)
            s_prime = env.curloc
            agent.update_table((s, a, r, s_prime), targetidx)
            s = s_prime
        agent.anneal_eps()
        print(f'에피소드 :{n_epi}, 액션 수: {acount}, 리턴: {cum_reward}, 성공율 : {(env.successcount/(env.successcount+env.failcount+0.00001)*100):2f} %')
        if n_epi%20 ==0:
            print(alst) #20회마다 액션리스트만 print

In [79]:
main()

에피소드 :0, 액션 수: 40, 리턴: 7968, 성공율 : 0.999990 %
[0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 3, 3, 3, 0, 0, 2, 1, 3, 3, 3, 0, 1, 3, 3, 3, 0, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1]
에피소드 :1, 액션 수: 32, 리턴: 7976, 성공율 : 0.999995 %
에피소드 :2, 액션 수: 32, 리턴: 7976, 성공율 : 0.999997 %
에피소드 :3, 액션 수: 36, 리턴: 7972, 성공율 : 0.999998 %
에피소드 :4, 액션 수: 38, 리턴: 6969, 성공율 : 0.999998 %
에피소드 :5, 액션 수: 34, 리턴: 7974, 성공율 : 0.999998 %
에피소드 :6, 액션 수: 38, 리턴: 6969, 성공율 : 0.999999 %
에피소드 :7, 액션 수: 40, 리턴: 7968, 성공율 : 0.999999 %
에피소드 :8, 액션 수: 40, 리턴: 6967, 성공율 : 0.999999 %
에피소드 :9, 액션 수: 40, 리턴: 7968, 성공율 : 0.999999 %
에피소드 :10, 액션 수: 36, 리턴: 7972, 성공율 : 0.999999 %
에피소드 :11, 액션 수: 38, 리턴: 7970, 성공율 : 0.999999 %
에피소드 :12, 액션 수: 38, 리턴: 7970, 성공율 : 0.999999 %
에피소드 :13, 액션 수: 38, 리턴: 7970, 성공율 : 0.999999 %
에피소드 :14, 액션 수: 40, 리턴: 7968, 성공율 : 0.999999 %
에피소드 :15, 액션 수: 36, 리턴: 7972, 성공율 : 0.999999 %
에피소드 :16, 액션 수: 40, 리턴: 7968, 성공율 : 0.999999 %
에피소드 :17, 액션 수: 42, 리턴: 7966, 성공율 : 0.999999 %
에피소드 :18, 액션 수: 36, 리턴: 7972, 성공율 : 0.99999

# 회고 + 더 나아가야 할 방향

예상한대로 완전히 수렴한 여러 개의 Q-Table의 성능은 완벽했다.

혹은 상태, 액션, 변수가 적다는 점에서 
MDP를 아는 상황을 상정해 수렴하는 상태가치 table만 알 수 있어도
더 빠른 최적 정책을 찾을 수 있을 것 같다.

목표가 더 많아지는 경우 각각의 목표에 따라 Q-Table을 만들어 해결할 수 있을 것으로 예상되지만

예측할 수 없는 변수들로 인해 전이 확률이 변화하거나 상태, 액션등이 많아서 테이블 전부 저장하기 힘들어진다면 단순 알고리즘보다 뉴럴넷을 사용하는 것이 연산량 대비 성능 비(?) 가 좋아지는 시점이 올 것으로 예상되지만 table기반 강화학습도 여전히 강력함을 확인 할 수 있었다.

이런 경우 뉴럴넷을 연결한다면 분리해서 학습 후 취합하는 방식과 하나의 뉴럴넷으로 피쳐들 간에 관계들을 파악하는 것 중 어느 것이 더 좋을지 모르겠다.

FC레이어들로 각각 학습시킨뒤에 선형결합으로 관계성을 파악하는 방식이 좋아보이는데...

##추가 계획

일단 일정 주기마다 업데이트한 Q-table을 validation set에 지속적으로 결과를 확인해보는 방식을 통해 적정 Q_table 수렴에 걸리는 episode와 수렴에 걸리는 시간, 사용된 램 용량 등을 체크해서 다른 분들의 모델과 비교해보는게 좋을 것 같다.

GPU를 쓰지않아도 속도가 그렇게 느린건 아니었으나 
정확한비교를 위해, GPU를 사용해서 학습시킬 수 있도록 해봐야할 것 같다.

또한 임의의 목표 A, B, C를 따로 잡고 Q러닝을 학습하는 것이 아닌 train set으로 학습시켰을때의 차이도 궁금하다.

## 궁금한 점들

보상을 몇으로 설정해주는게 가장 학습이 빠른지 궁금하다.
아마도... 선택할 액션의 숫자와 최적해(이동 수), grid의 가로, 세로 수에 따른 함수로 나타날 것으로 예상은 된다만,

만약 Target의 위치가 랜덤하게 생성된다면? 모든 State에 대한 학습을 하면 되지 않을까?

혹은 Target의 위치가 특정되지 않아 자유이동하는 물체라면 어떨까? 뜬구름 잡는건 말도안되니 5칸정도 이내에 trail을 남긴다면 어떤 방식으로 학습이 될까 궁금하다. 이런 경우라면 뉴럴넷이 더 유리할 수도 있을꺼같은데... 